In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [2]:
# CORE DATA MANIPULATION
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import math

# MACHINE LEARNING - SCIKIT-LEARN
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

# ADVANCED ML LIBRARIES
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# CONFIGURATION FOR PLOTS
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


# DEEP LEARNING 
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print("TensorFlow disponible")
except ImportError:
    print("TensorFlow no disponible en este entorno")


# UTILITIES
import os
import sys
import warnings
import itertools
from datetime import datetime, timedelta
import time
from collections import Counter
import pickle
import joblib

# JUPYTER SPECIFIC
from IPython.display import display, HTML
from tqdm.notebook import tqdm

# SUPPRESS WARNINGS
warnings.filterwarnings('ignore')

# PANDAS CONFIGURATION
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# NUMPY CONFIGURATION
np.random.seed(42)

# PLOTLY CONFIGURATION
pyo.init_notebook_mode(connected=True)

print("✅ Todas las librerías importadas correctamente!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn version: {__import__('sklearn').__version__}")
print(f"📈 Matplotlib version: {__import__('matplotlib').__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")

# ============================================================================
# FUNCIONES ÚTILES ADICIONALES
# ============================================================================

def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

def plot_missing_values(df):
    """Visualizar valores faltantes"""
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        plt.figure(figsize=(10, 6))
        missing.plot(kind='bar')
        plt.title('Valores Faltantes por Columna')
        plt.ylabel('Cantidad')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print("✅ No hay valores faltantes en el dataset")


2025-08-30 18:25:11.464770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756578311.743838      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756578311.836830      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow disponible


✅ Todas las librerías importadas correctamente!
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
🤖 Scikit-learn version: 1.2.2
📈 Matplotlib version: 3.7.2
🎨 Seaborn version: 0.12.2


In [3]:
#competencia https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/overview

# Files
# train.csv

# county - An ID code for the county.
# is_business - Boolean for whether or not the prosumer is a business.
# product_type - ID code with the following mapping of codes to contract types: {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}.
# target - The consumption or production amount for the relevant segment for the hour. The segments are defined by the county, is_business, and product_type.
# is_consumption - Boolean for whether or not this row's target is consumption or production.
# datetime - The Estonian time in EET (UTC+2) / EEST (UTC+3). It describes the start of the 1-hour period on which target is given.
# data_block_id - All rows sharing the same data_block_id will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather data_block_id for predictins made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.
# row_id - A unique identifier for the row.
# prediction_unit_id - A unique identifier for the county, is_business, and product_type combination. New prediction units can appear or disappear in the test set.
# gas_prices.csv

# origin_date - The date when the day-ahead prices became available.
# forecast_date - The date when the forecast prices should be relevant.
# [lowest/highest]_price_per_mwh - The lowest/highest price of natural gas that on the day ahead market that trading day, in Euros per megawatt hour equivalent.
# data_block_id
# client.csv

# product_type
# county - An ID code for the county. See county_id_to_name_map.json for the mapping of ID codes to county names.
# eic_count - The aggregated number of consumption points (EICs - European Identifier Code).
# installed_capacity - Installed photovoltaic solar panel capacity in kilowatts.
# is_business - Boolean for whether or not the prosumer is a business.
# date
# data_block_id
# electricity_prices.csv

# origin_date
# forecast_date - Represents the start of the 1-hour period when the price is valid
# euros_per_mwh - The price of electricity on the day ahead markets in euros per megawatt hour.
# data_block_id
# forecast_weather.csv Weather forecasts that would have been available at prediction time. Sourced from the European Centre for Medium-Range Weather Forecasts.

# [latitude/longitude] - The coordinates of the weather forecast.
# origin_datetime - The timestamp of when the forecast was generated.
# hours_ahead - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
# temperature - The air temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
# dewpoint - The dew point temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
# cloudcover_[low/mid/high/total] - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total. Estimated for the end of the 1-hour period.
# 10_metre_[u/v]_wind_component - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second. Estimated for the end of the 1-hour period.
# data_block_id
# forecast_datetime - The timestamp of the predicted weather. Generated from origin_datetime plus hours_ahead. This represents the start of the 1-hour period for which weather data are forecasted.
# direct_solar_radiation - The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the hour, in watt-hours per square meter.
# surface_solar_radiation_downwards - The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, accumulated during the hour, in watt-hours per square meter.
# snowfall - Snowfall over hour in units of meters of water equivalent.
# total_precipitation - The accumulated liquid, comprising rain and snow that falls on Earth's surface over the described hour, in units of meters.
# historical_weather.csv Historic weather data.

# datetime - This represents the start of the 1-hour period for which weather data are measured.
# temperature - Measured at the end of the 1-hour period.
# dewpoint - Measured at the end of the 1-hour period.
# rain - Different from the forecast conventions. The rain from large scale weather systems of the hour in millimeters.
# snowfall - Different from the forecast conventions. Snowfall over the hour in centimeters.
# surface_pressure - The air pressure at surface in hectopascals.
# cloudcover_[low/mid/high/total] - Different from the forecast conventions. Cloud cover at 0-3 km, 3-8, 8+, and total.
# windspeed_10m - Different from the forecast conventions. The wind speed at 10 meters above ground in meters per second.
# winddirection_10m - Different from the forecast conventions. The wind direction at 10 meters above ground in degrees.
# shortwave_radiation - Different from the forecast conventions. The global horizontal irradiation in watt-hours per square meter.
# direct_solar_radiation
# diffuse_radiation - Different from the forecast conventions. The diffuse solar irradiation in watt-hours per square meter.
# [latitude/longitude] - The coordinates of the weather station.
# data_block_id

In [4]:
#modelos a correr: NN por ser forecasting y algun xgb o lgb
#son 4 dias a predecir el comportamiento de los prosumers
#predecir con rolling de a 1 dia


#analizar y preparar
# como siempre primero ver los faltantes y los NaN y en cada caso decidir que hacer
# atento con valores expresados para el dia actual o la hora actual / la hora anterior / el dia de mañana , ver de no malinterpretar
# ver la correlacion de precios sea gas y/o electricidad para ver como se comparta con business y no business
# lo mismo con el clima para business y no business
# ver si aplica el supuesto de que los business si operan bajo esos valores (costos de electricidad/gas) y los no business no es tan relevante
# clima historical vs forecast , adaptar columnas que usan distintas proporciones , medir la certeza del forecast
# analizar por fuera como funcionan los paneles y como miden lo que miden ( para mejor entendimiento del df)

#columnas a agregar
#Datetime dias lu-1/ma-2/mi-3/ju-4/vi-5/sa-6/do-7 
#Date time horas y capaz minutos
#horario laboral ejemplo 7 a 17hs (analizarlo) sea en el dataset y por fuera
#horario business , para los que son business ver un pseudo horario de apertura y cierre como afecta y si se puede
#Dia 6hs a 18hs / noche de 18hs a 6hs ver si lo adapto con 4 columnas por las estaciones
#hora de dormir ejemplo 22hs a 6hs
#dias festivos y vacaciones , columnas binarias en ambos
#estaciones del año 4 columnas binarias
#periodogram para ver los lags y sea mensuales/quincenales/diarios/hora
#ver si es relevante agregar alguna columna con la poblacion mensual y usar un ratio (info a buscar afuera)

#sugerencias de features

##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico)
# Capacidad instalada per capita (installed_capacity / eic_count)
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

##Temporales:

# Lag de consumo/producción del mismo prediction_unit_id (24h, 48h, 168h)
# Media móvil de target por prediction_unit_id
# Cambios día a día (delta vs día anterior)

##Weather engineering:

# Índice de confort térmico (combinando temp + humidity)
# Potencial solar real (direct + diffuse radiation ajustado por cloudcover)
# Diferencia forecast vs historical weather (para medir accuracy del forecast)

##Segmentación:

# Clustering de prediction_units por comportamiento similar
# Ratios county-level (agregaciones por county vs individual)



In [5]:
#county_id_to_name_map.json
# {
#   "0": "HARJUMAA",
#   "1": "HIIUMAA",
#   "2": "IDA-VIRUMAA",
#   "3": "JÄRVAMAA",
#   "4": "JÕGEVAMAA",
#   "5": "LÄÄNE-VIRUMAA",
#   "6": "LÄÄNEMAA",
#   "7": "PÄRNUMAA",
#   "8": "PÕLVAMAA",
#   "9": "RAPLAMAA",
#   "10": "SAAREMAA",
#   "11": "TARTUMAA",
#   "12": "UNKNOWN",
#   "13": "VALGAMAA",
#   "14": "VILJANDIMAA",
#   "15": "VÕRUMAA"
# }

In [6]:
train                 = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")
client                = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/client.csv')
electricity_prices    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv')
forecast_weather      = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv')
gas_prices            = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv')
historical_weather    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv')
weather_station       = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')

test                  = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/test.csv')
client_t              = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv')
electricity_prices_t  = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity_prices.csv')
forecast_weather_t    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/forecast_weather.csv')
gas_prices_t          = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv')
revealed_targets      = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/revealed_targets.csv')



In [7]:
# pruebo pipeline

In [8]:
import pandas as pd
import numpy as np
import unicodedata
from sklearn.neighbors import NearestNeighbors
import gc
from collections import defaultdict
from pathlib import Path

class DataPreprocessingPipeline:
    """Pipeline modular para preprocessing de datos de train y test"""
    
    def __init__(self):
        self.county_id_to_name_map = {
            0: "HARJUMAA", 1: "HIIUMAA", 2: "IDA-VIRUMAA", 3: "JÄRVAMAA",
            4: "JÕGEVAMAA", 5: "LÄÄNE-VIRUMAA", 6: "LÄÄNEMAA", 7: "PÄRNUMAA",
            8: "PÕLVAMAA", 9: "RAPLAMAA", 10: "SAAREMAA", 11: "TARTUMAA",
            12: "UNKNOWN", 13: "VALGAMAA", 14: "VILJANDIMAA", 15: "VÕRUMAA"
        }
        self.county_name_to_id_map = {v: k for k, v in self.county_id_to_name_map.items()}
        
        # Diccionario de sinónimos para normalización
        self.synonyms_to_canonical = {
            "HARJU": "HARJUMAA", "HIIU": "HIIUMAA", "IDA-VIRU": "IDA-VIRUMAA",
            "JARVA": "JÄRVAMAA", "JÄRVA": "JÄRVAMAA", "JOGEVA": "JÕGEVAMAA", 
            "JÕGEVA": "JÕGEVAMAA", "LAANE-VIRU": "LÄÄNE-VIRUMAA", 
            "LÄÄNE-VIRU": "LÄÄNE-VIRUMAA", "LAANE": "LÄÄNEMAA", "LÄÄNE": "LÄÄNEMAA",
            "PARNU": "PÄRNUMAA", "PÄRNU": "PÄRNUMAA", "POLVA": "PÕLVAMAA", 
            "PÕLVA": "PÕLVAMAA", "RAPLA": "RAPLAMAA", "SAARE": "SAAREMAA",
            "TARTU": "TARTUMAA", "VALGA": "VALGAMAA", "VILJANDI": "VILJANDIMAA",
            "VORU": "VÕRUMAA", "VÕRU": "VÕRUMAA",
            # versiones ya canónicas
            **{name: name for name in self.county_id_to_name_map.values()}
        }
    
    def strip_accents(self, s: str) -> str:
        """Eliminar acentos de string"""
        return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    
    def normalize_county_name(self, x):
        """Normalizar nombre de county"""
        if pd.isna(x):
            return np.nan
        s = str(x).upper().strip()
        # eliminar sufijos frecuentes
        for suf in [" COUNTY", " MAAKOND"]:
            if s.endswith(suf):
                s = s[: -len(suf)]
        s = s.replace("  ", " ").replace("–", "-")
        s_noacc = self.strip_accents(s)
        s_noacc = s_noacc.replace("  ", " ")
        
        token = s_noacc.split()[0]
        if "-" in s_noacc:
            token = s_noacc
        
        # intentos de match
        if s_noacc in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[s_noacc]
        if token in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[token]
        if s in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[s]
        
        return np.nan
    
    def process_weather_stations(self, weather_station):
        """Procesar estaciones meteorológicas para asignar counties"""
        weather_station = weather_station.copy()
        
        # Normalizar nombres de county
        weather_station['county_name_norm'] = weather_station['county_name'].apply(self.normalize_county_name)
        
        # Si tengo 'county' numérico pero falta nombre, lo inferimos del ID
        mask = weather_station['county_name_norm'].isna() & weather_station['county'].notna()
        weather_station.loc[mask, 'county_name_norm'] = (
            weather_station.loc[mask, 'county'].astype(int).map(self.county_id_to_name_map)
        )
        
        # Completar faltantes por k-NN
        fcols = ['latitude', 'longitude']
        known = weather_station.dropna(subset=['county_name_norm']).copy()
        unknown = weather_station[weather_station['county_name_norm'].isna()].copy()
        
        if not unknown.empty and not known.empty:
            known_rad = np.radians(known[fcols].values)
            unknown_rad = np.radians(unknown[fcols].values)
            
            nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
            nbrs.fit(known_rad)
            distances, idxs = nbrs.kneighbors(unknown_rad)
            
            nn_names = known['county_name_norm'].values
            filled = []
            for neigh_idx in idxs:
                cands = nn_names[neigh_idx]
                vals, counts = np.unique(cands, return_counts=True)
                filled.append(vals[np.argmax(counts)])
            
            weather_station.loc[unknown.index, 'county_name_norm'] = filled
        
        # Sincronizar columnas finales
        weather_station['county_name'] = weather_station['county_name_norm']
        weather_station['county'] = weather_station['county_name'].map(self.county_name_to_id_map).astype('Int64')
        
        # Cualquier remanente a UNKNOWN (12)
        weather_station.loc[weather_station['county_name'].isna(), 'county'] = 12
        weather_station['county'] = weather_station['county'].astype('Int64')
        
        weather_station.drop(columns=['county_name_norm'], inplace=True)
        
        return weather_station
    
    def optimize_dtypes(self, df, is_train=True):
        """Optimizar tipos de datos para reducir memoria"""
        df = df.copy()
        
        # Tipos comunes
        if 'county' in df.columns:
            df['county'] = df['county'].astype('uint8')
        if 'is_business' in df.columns:
            df['is_business'] = df['is_business'].astype('uint8')
        if 'product_type' in df.columns:
            df['product_type'] = df['product_type'].astype('uint8')
        if 'is_consumption' in df.columns:
            df['is_consumption'] = df['is_consumption'].astype('uint8')
        if 'hour' in df.columns:
            df['hour'] = df['hour'].astype('uint8')
        
        # IDs
        if 'data_block_id' in df.columns:
            df['data_block_id'] = df['data_block_id'].astype('uint16')
        if 'row_id' in df.columns:
            df['row_id'] = df['row_id'].astype('uint32')
        if 'prediction_unit_id' in df.columns:
            df['prediction_unit_id'] = df['prediction_unit_id'].astype('uint8')
        
        # Coordenadas y variables meteorológicas - float32
        float_cols = ['latitude', 'longitude', 'target', 'lowest_price_per_mwh', 
                     'highest_price_per_mwh', 'euros_per_mwh', 'eic_count', 
                     'installed_capacity']
        
        # Variables meteorológicas
        weather_cols = ['ftemperature', 'fdewpoint', 'fcloudcover_high', 'fcloudcover_low', 
                       'fcloudcover_mid', 'fcloudcover_total', 'f10_metre_u_wind_component', 
                       'f10_metre_v_wind_component', 'fdirect_solar_radiation', 
                       'fsurface_solar_radiation_downwards', 'fsnowfall', 'ftotal_precipitation',
                       'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure',
                       'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high',
                       'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 
                       'direct_solar_radiation', 'diffuse_radiation']
        
        # Aplicar float32 a columnas que existen
        for col in float_cols + weather_cols:
            if col in df.columns:
                df[col] = df[col].astype('float32')
        
        # Weather forecast hour
        if 'weather_forecast_hour' in df.columns:
            df['weather_forecast_hour'] = df['weather_forecast_hour'].astype('uint8')
        
        return df


class TrainPipeline(DataPreprocessingPipeline):
    """Pipeline específico para datos de entrenamiento"""
    
    def __init__(self):
        super().__init__()
        self.temp_dir = Path("temp_chunks")
        self.temp_dir.mkdir(exist_ok=True)
    
    def part1_prepare_base_merges(self, train, gas_prices, electricity_prices, client):
        """Parte 1: Preparar datos base y hacer primeros merges"""
        print("=== PARTE 1: Preparación y merges base ===")
        
        # Preparar copias
        train1 = train.dropna().copy()
        gas_prices1 = gas_prices.copy()
        electricity_prices1 = electricity_prices.copy()
        client1 = client.copy()
        
        # Procesar train
        train1['datetime'] = pd.to_datetime(train1['datetime'])
        train1['hour'] = train1['datetime'].dt.hour
        train1['forecast_date'] = train1['datetime']
        
        # Merge 1: Gas prices
        print("Procesando gas prices...")
        gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
        gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
        gas_prices1 = gas_prices1.rename(columns={'origin_date': 'gas_origin_date'})
        
        # Expandir gas a nivel horario
        gas_hourly = []
        for _, row in gas_prices1.iterrows():
            for hour in range(24):
                new_row = {
                    'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
                    'lowest_price_per_mwh': row['lowest_price_per_mwh'],
                    'highest_price_per_mwh': row['highest_price_per_mwh']
                }
                gas_hourly.append(new_row)
        
        gas_prices1_hourly = pd.DataFrame(gas_hourly)
        train2 = pd.merge(train1, gas_prices1_hourly, on='forecast_date', how='left')
        
        # Forward fill gas prices
        train2['lowest_price_per_mwh'] = train2['lowest_price_per_mwh'].fillna(method='ffill')
        train2['highest_price_per_mwh'] = train2['highest_price_per_mwh'].fillna(method='ffill')
        
        # Merge 2: Electricity prices
        print("Procesando electricity prices...")
        electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
        electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])
        
        elec_columns = ['forecast_date', 'euros_per_mwh']
        train3 = pd.merge(train2, electricity_prices1[elec_columns], on='forecast_date', how='left')
        train3['euros_per_mwh'] = train3['euros_per_mwh'].fillna(method='ffill')
        
        # Merge 3: Client data
        print("Procesando client data...")
        client1['date'] = pd.to_datetime(client1['date']).dt.date
        
        train4 = train3.merge(client1.drop('data_block_id', axis=1),
                             left_on=['product_type', 'county', 'is_business', train3['datetime'].dt.date],
                             right_on=['product_type', 'county', 'is_business', 'date'],
                             how='left')
        
        train4 = train4.drop('date', axis=1)
        train4 = train4.rename(columns={'data_block_id_x': 'data_block_id'})
        
        # Forward fill client data
        print("Completando datos de clientes...")
        train4 = train4.sort_values(['county', 'is_business', 'product_type', 'datetime'])
        train4['eic_count'] = train4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
        train4['installed_capacity'] = train4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')
        
        # Rellenar restantes con promedio por county
        train4['eic_count'] = train4['eic_count'].fillna(train4.groupby('county')['eic_count'].transform('mean'))
        train4['installed_capacity'] = train4['installed_capacity'].fillna(train4.groupby('county')['installed_capacity'].transform('mean'))
        
        # Optimizar tipos
        train4 = self.optimize_dtypes(train4, is_train=True)
        
        print(f"Parte 1 completada. Dataset: {train4.shape[0]:,} filas, {train4.shape[1]} columnas")
        return train4
    
    def part2_prepare_weather_merge(self, train4, forecast_weather, historical_weather, weather_station):
        """Parte 2: Preparar datos meteorológicos y hacer merge"""
        print("=== PARTE 2: Preparación datos meteorológicos ===")
        
        # Procesar weather stations
        weather_station_processed = self.process_weather_stations(weather_station)
        
        # Procesar forecast weather
        print("Procesando forecast weather...")
        forecast_weather1 = forecast_weather.dropna().copy()
        
        # Merge forecast con weather stations
        forecast_weather1 = forecast_weather1.merge(
            weather_station_processed[['latitude', 'longitude', 'county', 'county_name']], 
            on=['latitude', 'longitude'], 
            how='left'
        )
        
        # Completar counties faltantes con k-NN
        missing_county = forecast_weather1['county'].isna()
        if missing_county.sum() > 0:
            print(f"Completando {missing_county.sum()} counties faltantes con k-NN...")
            known = weather_station_processed[['latitude', 'longitude', 'county', 'county_name']].copy()
            missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
            
            nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
            nbrs.fit(known[['latitude', 'longitude']].values)
            distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
            
            nearest_counties = known.iloc[indices.flatten()]
            forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
            forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values
        
        # Transformar forecast weather
        forecast_weather2 = forecast_weather1.copy()
        forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)
        forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])
        
        # Renombrar con prefijo 'f'
        rename_dict = {}
        for col in forecast_weather2.columns:
            if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                          'forecast_datetime']:
                rename_dict[col] = 'f' + col
        
        forecast_weather2 = forecast_weather2.rename(columns=rename_dict)
        forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead": "weather_forecast_hour"})
        
        # Merge con historical weather
        print("Mergeando forecast con historical weather...")
        historical_weather1 = historical_weather.copy()
        historical_weather1['datetime'] = pd.to_datetime(historical_weather1['datetime'])
        
        forecast_weather2 = forecast_weather2.rename(columns={'data_block_id': 'f_data_block_id'})
        
        merged_weather = pd.merge(
            forecast_weather2,
            historical_weather1,
            left_on=['latitude', 'longitude', 'forecast_datetime'],
            right_on=['latitude', 'longitude', 'datetime'],
            how='inner'
        )
        
        # Limpiar y optimizar
        merged_weather = merged_weather.drop(['f_data_block_id', 'forecast_datetime'], axis=1)
        merged_weather = self.optimize_dtypes(merged_weather, is_train=True)
        
        print(f"Weather data preparado: {merged_weather.shape[0]:,} filas, {merged_weather.shape[1]} columnas")
        return train4, merged_weather
    
    def part3_final_merge_and_cleanup(self, train4, merged_weather, chunk_size=1_000_000):
        """Parte 3: Merge final y limpieza"""
        print("=== PARTE 3: Merge final y limpieza ===")
        
        # Definir períodos para procesar por chunks
        periods = [
            ('2021-09-01', '2021-10-01'), ('2021-10-01', '2021-11-01'), ('2021-11-01', '2021-12-01'), 
            ('2021-12-01', '2022-01-01'), ('2022-01-01', '2022-02-01'), ('2022-02-01', '2022-03-01'), 
            ('2022-03-01', '2022-04-01'), ('2022-04-01', '2022-05-01'), ('2022-05-01', '2022-06-01'), 
            ('2022-06-01', '2022-07-01'), ('2022-07-01', '2022-08-01'), ('2022-08-01', '2022-09-01'),
            ('2022-09-01', '2022-10-01'), ('2022-10-01', '2022-11-01'), ('2022-11-01', '2022-12-01'), 
            ('2022-12-01', '2023-01-01'), ('2023-01-01', '2023-02-01'), ('2023-02-01', '2023-03-01'), 
            ('2023-03-01', '2023-04-01'), ('2023-04-01', '2023-05-01'), ('2023-05-01', '2023-05-31')
        ]
        
        # Procesar por chunks temporales
        chunk_files = []
        for i, (start_date, end_date) in enumerate(periods):
            print(f"Procesando período {i+1}/{len(periods)}: {start_date} a {end_date}")
            
            # Filtrar por período
            mask_train = (train4['datetime'] >= start_date) & (train4['datetime'] < end_date)
            train4_chunk = train4[mask_train].copy()
            
            mask_weather = (merged_weather['datetime'] >= start_date) & (merged_weather['datetime'] < end_date)
            weather_chunk = merged_weather[mask_weather].copy()
            
            if len(train4_chunk) > 0 and len(weather_chunk) > 0:
                # Merge chunk
                chunk_result = pd.merge(train4_chunk, weather_chunk, 
                               on=['county', 'datetime'], 
                               how='left')
                
                # Guardar chunk temporal
                chunk_file = self.temp_dir / f'train5_chunk_{i+1}.parquet'
                chunk_result.to_parquet(chunk_file, index=False)
                chunk_files.append(chunk_file)
                
                print(f"Chunk {i+1} guardado: {chunk_result.shape}")
                
                # Liberar memoria
                del train4_chunk, weather_chunk, chunk_result
                gc.collect()
        
        # Concatenar chunks
        print("Concatenando chunks...")
        final_chunks = []
        for chunk_file in chunk_files:
            chunk_data = pd.read_parquet(chunk_file)
            final_chunks.append(chunk_data)
        
        train5 = pd.concat(final_chunks, ignore_index=True)
        
        # Limpieza final
        print("Aplicando limpieza final...")
        train5 = self._cleanup_merged_data(train5, chunk_size)
        
        # Limpiar archivos temporales
        for chunk_file in chunk_files:
            chunk_file.unlink()
        
        return train5
    
    def _cleanup_merged_data(self, train5, chunk_size=1_000_000):
        """Limpiar datos después del merge"""
        # Identificar columnas a eliminar y renombrar
        sample = train5.head(1000)
        cols_to_drop = [col for col in sample.columns if col.endswith('_y')]
        rename_dict = {col: col[:-2] for col in sample.columns if col.endswith('_x')}
        
        print(f"Eliminando {len(cols_to_drop)} columnas, renombrando {len(rename_dict)}")
        
        # Procesar por chunks
        chunks_processed = []
        total_rows = len(train5)
        n_chunks = (total_rows // chunk_size) + 1
        
        for i in range(n_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            print(f"Procesando chunk limpieza {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
            
            chunk = train5.iloc[start_idx:end_idx].copy()
            
            # Filtrar county != 12
            chunk = chunk[chunk['county'] != 12]
            
            if len(chunk) > 0:
                # Eliminar columnas _y y renombrar _x
                chunk = chunk.drop(columns=cols_to_drop)
                chunk.rename(columns=rename_dict, inplace=True)
                chunks_processed.append(chunk)
            
            del chunk
            gc.collect()
        
        # Concatenar final
        train5_clean = pd.concat(chunks_processed, ignore_index=True)
        del chunks_processed
        gc.collect()
        
        # Eliminar datos de clima faltantes
        weather_key_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']
        initial_rows = len(train5_clean)
        train5_clean = train5_clean.dropna(subset=weather_key_cols)
        print(f"Filas eliminadas por clima faltante: {initial_rows - len(train5_clean):,}")
        
        # Reordenar columnas
        train5_clean = self._reorder_columns(train5_clean)
        
        return train5_clean
    
    def _reorder_columns(self, df):
        """Reordenar columnas en el orden especificado"""
        main_cols = [
            'row_id', 'target', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
            'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
            'longitude', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 
            'eic_count', 'installed_capacity'
        ]
        
        weather_pairs = [
            'ftemperature', 'temperature', 'fdewpoint', 'dewpoint',
            'fcloudcover_high', 'cloudcover_high', 'fcloudcover_low', 'cloudcover_low', 
            'fcloudcover_mid', 'cloudcover_mid', 'fcloudcover_total', 'cloudcover_total',
            'f10_metre_u_wind_component', 'windspeed_10m', 'f10_metre_v_wind_component', 
            'winddirection_10m', 'fdirect_solar_radiation', 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards', 'shortwave_radiation', 'diffuse_radiation',
            'fsnowfall', 'snowfall', 'ftotal_precipitation', 'rain', 'surface_pressure', 
            'weather_forecast_hour'
        ]
        
        final_order = main_cols.copy()
        for col in weather_pairs:
            if col in df.columns:
                final_order.append(col)
        
        remaining_cols = [col for col in df.columns if col not in final_order]
        final_order.extend(remaining_cols)
        
        return df[final_order]
    
    def run_full_pipeline(self, train, gas_prices, electricity_prices, client, 
                         forecast_weather, historical_weather, weather_station):
        """Ejecutar pipeline completo de entrenamiento"""
        print("Iniciando pipeline completo de entrenamiento...")
        
        # Parte 1: Merges base
        train4 = self.part1_prepare_base_merges(train, gas_prices, electricity_prices, client)
        
        # Parte 2: Preparar datos meteorológicos
        train4, merged_weather = self.part2_prepare_weather_merge(
            train4, forecast_weather, historical_weather, weather_station
        )
        
        # Parte 3: Merge final y limpieza
        train5 = self.part3_final_merge_and_cleanup(train4, merged_weather)
        
        print(f"Pipeline completado! Dataset final: {train5.shape[0]:,} filas, {train5.shape[1]} columnas")
        return train5


class TestPipeline(DataPreprocessingPipeline):
    """Pipeline simplificado para datos de test"""
    
    def run_test_pipeline(self, test, gas_prices_t, electricity_prices_t, client_t, 
                         forecast_weather_t, weather_station):
        """Pipeline completo para test set (más simple, sin historical weather)"""
        print("=== PIPELINE TEST ===")
        
        # Preparar datos base
        test1 = test.copy()
        test1['datetime'] = pd.to_datetime(test1['datetime'])
        test1['hour'] = test1['datetime'].dt.hour
        test1['forecast_date'] = test1['datetime']
        
        # Merge gas prices
        print("Procesando gas prices...")
        gas_prices1 = gas_prices_t.copy()
        gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
        gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
        
        gas_hourly = []
        for _, row in gas_prices1.iterrows():
            for hour in range(24):
                new_row = {
                    'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
                    'lowest_price_per_mwh': row['lowest_price_per_mwh'],
                    'highest_price_per_mwh': row['highest_price_per_mwh']
                }
                gas_hourly.append(new_row)
        
        gas_prices_hourly = pd.DataFrame(gas_hourly)
        test2 = pd.merge(test1, gas_prices_hourly, on='forecast_date', how='left')
        test2['lowest_price_per_mwh'] = test2['lowest_price_per_mwh'].fillna(method='ffill')
        test2['highest_price_per_mwh'] = test2['highest_price_per_mwh'].fillna(method='ffill')
        
        # Merge electricity prices
        print("Procesando electricity prices...")
        electricity_prices1 = electricity_prices_t.copy()
        electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
        electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])
        
        test3 = pd.merge(test2, electricity_prices1[['forecast_date', 'euros_per_mwh']], 
                        on='forecast_date', how='left')
        test3['euros_per_mwh'] = test3['euros_per_mwh'].fillna(method='ffill')
        
        # Merge client data
        print("Procesando client data...")
        client1 = client_t.copy()
        client1['date'] = pd.to_datetime(client1['date']).dt.date
        
        test4 = test3.merge(client1.drop('data_block_id', axis=1),
                           left_on=['product_type', 'county', 'is_business', test3['datetime'].dt.date],
                           right_on=['product_type', 'county', 'is_business', 'date'],
                           how='left')
        
        test4 = test4.drop('date', axis=1)
        test4 = test4.rename(columns={'data_block_id_x': 'data_block_id'})
        
        # Forward fill client data
        test4 = test4.sort_values(['county', 'is_business', 'product_type', 'datetime'])
        test4['eic_count'] = test4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
        test4['installed_capacity'] = test4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')
        
        # Rellenar restantes con promedio por county
        test4['eic_count'] = test4['eic_count'].fillna(test4.groupby('county')['eic_count'].transform('mean'))
        test4['installed_capacity'] = test4['installed_capacity'].fillna(test4.groupby('county')['installed_capacity'].transform('mean'))
        
        # Procesar forecast weather
        print("Procesando forecast weather...")
        weather_station_processed = self.process_weather_stations(weather_station)
        
        forecast_weather1 = forecast_weather_t.copy()
        
        # Merge forecast con weather stations
        forecast_weather1 = forecast_weather1.merge(
            weather_station_processed[['latitude', 'longitude', 'county', 'county_name']], 
            on=['latitude', 'longitude'], 
            how='left'
        )
        
        # Completar counties faltantes con k-NN
        missing_county = forecast_weather1['county'].isna()
        if missing_county.sum() > 0:
            print(f"Completando {missing_county.sum()} counties faltantes con k-NN...")
            known = weather_station_processed[['latitude', 'longitude', 'county', 'county_name']].copy()
            missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
            
            nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
            nbrs.fit(known[['latitude', 'longitude']].values)
            distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
            
            nearest_counties = known.iloc[indices.flatten()]
            forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
            forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values
        
        # Transformar forecast weather
        forecast_weather2 = forecast_weather1.copy()
        forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)
        forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])
        
        # Renombrar con prefijo 'f'
        rename_dict = {}
        for col in forecast_weather2.columns:
            if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                          'forecast_datetime']:
                rename_dict[col] = 'f' + col
        
        forecast_weather2 = forecast_weather2.rename(columns=rename_dict)
        forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead": "weather_forecast_hour"})
        
        # Para test, no tenemos historical weather, así que creamos columnas dummy o usamos solo forecast
        # Opción 1: Solo usar forecast weather (renombrar las f columns)
        # Opción 2: Crear columnas históricas como NaN y llenar después
        
        # Vamos con opción 1: usar forecast como histórico también (aproximación)
        historical_cols_mapping = {
            'ftemperature': 'temperature',
            'fdewpoint': 'dewpoint', 
            'fcloudcover_high': 'cloudcover_high',
            'fcloudcover_low': 'cloudcover_low',
            'fcloudcover_mid': 'cloudcover_mid', 
            'fcloudcover_total': 'cloudcover_total',
            'fdirect_solar_radiation': 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards': 'shortwave_radiation',
            'fsnowfall': 'snowfall',
            'ftotal_precipitation': 'rain'
        }
        
        # Crear versiones históricas basadas en forecast
        for fcol, hcol in historical_cols_mapping.items():
            if fcol in forecast_weather2.columns:
                forecast_weather2[hcol] = forecast_weather2[fcol]
        
        # Agregar columnas que solo existen en historical
        forecast_weather2['surface_pressure'] = 1013.25  # valor típico
        forecast_weather2['windspeed_10m'] = np.sqrt(
            forecast_weather2['f10_metre_u_wind_component']**2 + 
            forecast_weather2['f10_metre_v_wind_component']**2
        ) if 'f10_metre_u_wind_component' in forecast_weather2.columns else 0
        
        forecast_weather2['winddirection_10m'] = np.arctan2(
            forecast_weather2['f10_metre_v_wind_component'], 
            forecast_weather2['f10_metre_u_wind_component']
        ) * 180 / np.pi if 'f10_metre_v_wind_component' in forecast_weather2.columns else 0
        
        forecast_weather2['diffuse_radiation'] = forecast_weather2.get('fsurface_solar_radiation_downwards', 0) - forecast_weather2.get('fdirect_solar_radiation', 0)
        forecast_weather2['diffuse_radiation'] = forecast_weather2['diffuse_radiation'].clip(lower=0)
        
        # Limpiar y optimizar
        forecast_weather2 = forecast_weather2.drop(columns=['data_block_id'], errors='ignore')
        forecast_weather2 = self.optimize_dtypes(forecast_weather2, is_train=False)
        
        # Merge final con weather
        print("Merge final con datos meteorológicos...")
        test5 = pd.merge(test4, forecast_weather2, 
                        left_on=['datetime', 'county'], 
                        right_on=['forecast_datetime', 'county'], 
                        how='left')
        
        # Limpiar columnas duplicadas y renombrar
        cols_to_drop = [col for col in test5.columns if col.endswith('_y')]
        rename_dict = {col: col[:-2] for col in test5.columns if col.endswith('_x')}
        
        test5 = test5.drop(columns=cols_to_drop + ['forecast_datetime'], errors='ignore')
        test5.rename(columns=rename_dict, inplace=True)
        
        # Optimizar tipos finales
        test5 = self.optimize_dtypes(test5, is_train=False)
        
        # Reordenar columnas igual que train
        test5 = self._reorder_columns_test(test5)
        
        print(f"Pipeline test completado! Dataset: {test5.shape[0]:,} filas, {test5.shape[1]} columnas")
        return test5
    
    def _reorder_columns_test(self, df):
        """Reordenar columnas para test (mismo orden que train)"""
        main_cols = [
            'row_id', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
            'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
            'longitude', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 
            'eic_count', 'installed_capacity'
        ]
        
        weather_pairs = [
            'ftemperature', 'temperature', 'fdewpoint', 'dewpoint',
            'fcloudcover_high', 'cloudcover_high', 'fcloudcover_low', 'cloudcover_low', 
            'fcloudcover_mid', 'cloudcover_mid', 'fcloudcover_total', 'cloudcover_total',
            'f10_metre_u_wind_component', 'windspeed_10m', 'f10_metre_v_wind_component', 
            'winddirection_10m', 'fdirect_solar_radiation', 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards', 'shortwave_radiation', 'diffuse_radiation',
            'fsnowfall', 'snowfall', 'ftotal_precipitation', 'rain', 'surface_pressure', 
            'weather_forecast_hour'
        ]
        
        final_order = []
        # Solo agregar columnas que existen
        for col in main_cols:
            if col in df.columns:
                final_order.append(col)
        
        for col in weather_pairs:
            if col in df.columns:
                final_order.append(col)
        
        remaining_cols = [col for col in df.columns if col not in final_order]
        final_order.extend(remaining_cols)
        
        return df[final_order]


# Funciones de utilidad para usar el pipeline
def run_train_pipeline(train, gas_prices, electricity_prices, client, 
                      forecast_weather, historical_weather, weather_station):
    """
    Función principal para ejecutar pipeline de entrenamiento
    
    Returns:
        pd.DataFrame: Dataset procesado train5
    """
    pipeline = TrainPipeline()
    return pipeline.run_full_pipeline(
        train, gas_prices, electricity_prices, client, 
        forecast_weather, historical_weather, weather_station
    )

def run_test_pipeline(test, gas_prices_t, electricity_prices_t, client_t, 
                     forecast_weather_t, weather_station):
    """
    Función principal para ejecutar pipeline de test
    
    Returns:
        pd.DataFrame: Dataset procesado test5
    """
    pipeline = TestPipeline()
    return pipeline.run_test_pipeline(
        test, gas_prices_t, electricity_prices_t, client_t, 
        forecast_weather_t, weather_station
    )

# Ejemplo de uso:
"""
# Para train:
train5 = run_train_pipeline(
    train, gas_prices, electricity_prices, client, 
    forecast_weather, historical_weather, weather_station
)

# Para test:
test5 = run_test_pipeline(
    test, gas_prices_t, electricity_prices_t, client_t, 
    forecast_weather_t, weather_station
)
"""

'\n# Para train:\ntrain5 = run_train_pipeline(\n    train, gas_prices, electricity_prices, client, \n    forecast_weather, historical_weather, weather_station\n)\n\n# Para test:\ntest5 = run_test_pipeline(\n    test, gas_prices_t, electricity_prices_t, client_t, \n    forecast_weather_t, weather_station\n)\n'

In [9]:
train5 = run_train_pipeline(
    train, gas_prices, electricity_prices, client, 
    forecast_weather, historical_weather, weather_station
)

Iniciando pipeline completo de entrenamiento...
=== PARTE 1: Preparación y merges base ===
Procesando gas prices...
Procesando electricity prices...
Procesando client data...
Completando datos de clientes...
Parte 1 completada. Dataset: 2,017,824 filas, 16 columnas
=== PARTE 2: Preparación datos meteorológicos ===
Procesando forecast weather...
Completando 2140318 counties faltantes con k-NN...
Mergeando forecast con historical weather...
Weather data preparado: 3,418,242 filas, 32 columnas
=== PARTE 3: Merge final y limpieza ===
Procesando período 1/21: 2021-09-01 a 2021-10-01
Chunk 1 guardado: (1423080, 46)
Procesando período 2/21: 2021-10-01 a 2021-11-01
Chunk 2 guardado: (1532066, 46)
Procesando período 3/21: 2021-11-01 a 2021-12-01
Chunk 3 guardado: (1484640, 46)
Procesando período 4/21: 2021-12-01 a 2022-01-01
Chunk 4 guardado: (1543056, 46)
Procesando período 5/21: 2022-01-01 a 2022-02-01
Chunk 5 guardado: (1590672, 46)
Procesando período 6/21: 2022-02-01 a 2022-03-01
Chunk 6 gu

In [10]:
train5.head()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption
6,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.099998,24.200001,45.23,46.32,86.879997,108.0,952.890015,12.681543,12.4,9.783228,9.7,0.023590,11.0,0.002380,10.0,0.001251,0.0,0.026398,12.0,1.840991,4.222222,-3.857846,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1009.200012,1.0,0
7,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.099998,25.200001,45.23,46.32,86.879997,108.0,952.890015,12.868921,12.3,9.498316,9.6,0.431854,6.0,0.211182,23.0,0.006790,1.0,0.548508,23.0,1.505298,4.027778,-3.590024,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1004.200012,1.0,0
8,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,22.700001,45.23,46.32,86.879997,108.0,952.890015,15.041773,15.2,11.860376,11.8,0.134674,18.0,0.202515,7.0,0.003906,0.0,0.308930,12.0,3.185351,9.055555,-8.173276,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.000000,1.0,0
9,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,23.200001,45.23,46.32,86.879997,108.0,952.890015,14.632105,14.9,11.773584,11.5,0.255188,19.0,0.036774,6.0,0.026245,0.0,0.286194,11.0,3.474780,8.361111,-8.008969,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1014.500000,1.0,0
10,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,23.700001,45.23,46.32,86.879997,108.0,952.890015,14.480005,12.8,11.581568,10.4,0.338074,12.0,0.037109,4.0,0.016510,1.0,0.368134,8.0,3.211841,5.416667,-7.426206,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1014.000000,1.0,0


In [12]:
train5.isnull().sum()

row_id                                0
target                                0
datetime                              0
forecast_date                         0
hour                                  0
data_block_id                         0
prediction_unit_id                    0
is_business                           0
product_type                          0
county                                0
latitude                              0
longitude                             0
lowest_price_per_mwh                  0
highest_price_per_mwh                 0
euros_per_mwh                         0
eic_count                             0
installed_capacity                    0
ftemperature                          0
temperature                           0
fdewpoint                             0
dewpoint                              0
fcloudcover_high                      0
cloudcover_high                       0
fcloudcover_low                       0
cloudcover_low                        0


In [11]:
a

NameError: name 'a' is not defined

In [ ]:
#target 528 nulos de mas de 2kk , los omito
#fecha inicio 2021-09-01
#Dataset Shape: (2018352, 9)
#🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id']
#📝 Columnas categóricas: ['datetime']
#quick_info(train)
#train.head()
#train.describe(include="all")

In [ ]:
#train1 = train.dropna()
#quick_info(train1)

In [ ]:
# # 0 nulos
#fecha inicio 2021-09-01
# Dataset Shape: (41919, 7)
#🔢 Columnas numéricas: ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'data_block_id']
#📝 Columnas categóricas: ['date']
#quick_info(client)
#client.head()
# client.describe(include="all")

In [ ]:
# # 0 nulos
#fecha inicio 2021-08-31
#📊 Dataset Shape: (15286, 4)
#🔢 Columnas numéricas: ['euros_per_mwh', 'data_block_id']
#📝 Columnas categóricas: ['forecast_date', 'origin_date']
#quick_info(electricity_prices)
#electricity_prices.head()
#electricity_prices.describe(include="all")
# euros_per_mwh porque el minimo es -10.06 ?
# un supuesto seria que es lo que paga la empresa si aportas electricidad , pero siendo el max 4000 , hace ruido pensar eso

In [ ]:
# 2 nulos en 'surface_solar_radiation_downwards' , los omito
#fecha inicio 2021-09-01
#📊 Dataset Shape: (3424512, 18)
#🔢 Columnas numéricas: ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid',
#'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'data_block_id', 'direct_solar_radiation',
#'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
#📝 Columnas categóricas: ['origin_datetime', 'forecast_datetime']
#quick_info(forecast_weather)
#forecast_weather.head()
#forecast_weather.describe(include="all")


In [ ]:
#forecast_weather1 = forecast_weather.dropna()
#quick_info(forecast_weather1)

In [ ]:
#0 nulos
#fecha inicio 2021-09-01
#📊 Dataset Shape: (637, 5)
#🔢 Columnas numéricas: ['lowest_price_per_mwh', 'highest_price_per_mwh', 'data_block_id']
#📝 Columnas categóricas: ['forecast_date', 'origin_date']
#quick_info(gas_prices)
#gas_prices.head()
#gas_prices.describe(include="all")

In [ ]:
#0 nulos
#fecha inicio 2021-09-01
#📊 Dataset Shape: (1710802, 18)
#🔢 Columnas numéricas: ['temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure', 'cloudcover_total', 'cloudcover_low',
#'cloudcover_mid', 'cloudcover_high', 'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
#'diffuse_radiation', 'latitude', 'longitude', 'data_block_id']
#📝 Columnas categóricas: ['datetime']
#quick_info(historical_weather)
#historical_weather.head()
#historical_weather.describe(include="all")

In [ ]:
# ver si completo la info por fuera de los county_name
# county_name 63 nulos / county 63 nulos
# 📊 Dataset Shape: (112, 4)
# 🔢 Columnas numéricas: ['longitude', 'latitude', 'county']
# 📝 Columnas categóricas: ['county_name']
#quick_info(weather_station)
#weather_station.head(50)
#weather_station.describe(include="all")

In [ ]:
weather_station1 = weather_station.copy()


In [ ]:
import unicodedata
from sklearn.neighbors import NearestNeighbors

# === 1) Diccionarios canonicos ===
county_id_to_name_map = {
    0: "HARJUMAA", 1: "HIIUMAA", 2: "IDA-VIRUMAA", 3: "JÄRVAMAA",
    4: "JÕGEVAMAA", 5: "LÄÄNE-VIRUMAA", 6: "LÄÄNEMAA", 7: "PÄRNUMAA",
    8: "PÕLVAMAA", 9: "RAPLAMAA", 10: "SAAREMAA", 11: "TARTUMAA",
    12: "UNKNOWN", 13: "VALGAMAA", 14: "VILJANDIMAA", 15: "VÕRUMAA"
}
county_name_to_id_map = {v: k for k, v in county_id_to_name_map.items()}

# Para normalizar entradas (sin/ con acentos, "County"/"maakond", etc.)
synonyms_to_canonical = {
    # clave "base" (sin acentos/variantes) -> canon con acentos
    "HARJU": "HARJUMAA",
    "HIIU": "HIIUMAA",
    "IDA-VIRU": "IDA-VIRUMAA",
    "JARVA": "JÄRVAMAA", "JÄRVA": "JÄRVAMAA",
    "JOGEVA": "JÕGEVAMAA", "JÕGEVA": "JÕGEVAMAA",
    "LAANE-VIRU": "LÄÄNE-VIRUMAA", "LÄÄNE-VIRU": "LÄÄNE-VIRUMAA",
    "LAANE": "LÄÄNEMAA", "LÄÄNE": "LÄÄNEMAA",
    "PARNU": "PÄRNUMAA", "PÄRNU": "PÄRNUMAA",
    "POLVA": "PÕLVAMAA", "PÕLVA": "PÕLVAMAA",
    "RAPLA": "RAPLAMAA",
    "SAARE": "SAAREMAA",
    "TARTU": "TARTUMAA",
    "VALGA": "VALGAMAA",
    "VILJANDI": "VILJANDIMAA",
    "VORU": "VÕRUMAA", "VÕRU": "VÕRUMAA",

    # versiones ya canónicas
    "HARJUMAA": "HARJUMAA",
    "HIIUMAA": "HIIUMAA",
    "IDA-VIRUMAA": "IDA-VIRUMAA",
    "JÄRVAMAA": "JÄRVAMAA",
    "JÕGEVAMAA": "JÕGEVAMAA",
    "LÄÄNE-VIRUMAA": "LÄÄNE-VIRUMAA",
    "LÄÄNEMAA": "LÄÄNEMAA",
    "PÄRNUMAA": "PÄRNUMAA",
    "PÕLVAMAA": "PÕLVAMAA",
    "RAPLAMAA": "RAPLAMAA",
    "SAAREMAA": "SAAREMAA",
    "TARTUMAA": "TARTUMAA",
    "VALGAMAA": "VALGAMAA",
    "VILJANDIMAA": "VILJANDIMAA",
    "VÕRUMAA": "VÕRUMAA",
}

def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))

def normalize_county_name(x):
    if pd.isna(x):
        return np.nan
    s = str(x).upper().strip()
    # eliminar sufijos frecuentes
    for suf in [" COUNTY", " MAAKOND"]:
        if s.endswith(suf):
            s = s[: -len(suf)]
    s = s.replace("  ", " ").replace("–", "-")  # guion raro -> normal
    s_noacc = strip_accents(s)
    s_noacc = s_noacc.replace("  ", " ")
    # prueba de match exacto o por prefijo lógico
    # ej. "HARJU", "HARJU MAAKOND" -> "HARJU"
    token = s_noacc.split()[0]  # primer palabra alcanza en estos casos
    # casos con guion (IDA-VIRU, LAA NE-VIRU, etc.)
    if "-" in s_noacc:
        token = s_noacc  # mantener completo en compuestos

    # intento 1: match directo con cadena completa sin acentos
    if s_noacc in synonyms_to_canonical:
        return synonyms_to_canonical[s_noacc]
    # intento 2: match por token/prefijo conocido
    if token in synonyms_to_canonical:
        return synonyms_to_canonical[token]
    # intento 3: si ya vino canónico exacto (con acentos)
    if s in synonyms_to_canonical:
        return synonyms_to_canonical[s]

    return np.nan  # no lo reconozco

# === 2) Normalizar lo existente sin romper los NaN ===
# (NO convertir NaN a "NAN")
weather_station1['county_name_norm'] = weather_station1['county_name'].apply(normalize_county_name)

# Si tengo 'county' numérico pero falta nombre, lo inferimos del ID
mask = weather_station1['county_name_norm'].isna() & weather_station1['county'].notna()
weather_station1.loc[mask, 'county_name_norm'] = (
    weather_station1.loc[mask, 'county'].astype(int).map(county_id_to_name_map)
)

# === 3) Completar faltantes por k-NN (lat/lon) ===
fcols = ['latitude', 'longitude']
known = weather_station1.dropna(subset=['county_name_norm']).copy()
unknown = weather_station1[weather_station1['county_name_norm'].isna()].copy()

if not unknown.empty and not known.empty:
    # k-NN en (lat, lon) usando Haversine (necesita radianes)
    # ojo: haversine en sklearn requiere [lat, lon] en radianes
    known_rad = np.radians(known[fcols].values)
    unknown_rad = np.radians(unknown[fcols].values)

    nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
    nbrs.fit(known_rad)
    distances, idxs = nbrs.kneighbors(unknown_rad)

    # votación simple de los 3 vecinos (majority vote)
    nn_names = known['county_name_norm'].values
    filled = []
    for neigh_idx in idxs:
        cands = nn_names[neigh_idx]
        # mayoría; en empate, tomar el 1er vecino
        vals, counts = np.unique(cands, return_counts=True)
        filled.append(vals[np.argmax(counts)])

    weather_station1.loc[unknown.index, 'county_name_norm'] = filled

# === 4) Sincronizar columnas finales ===
weather_station1['county_name'] = weather_station1['county_name_norm']
weather_station1['county'] = weather_station1['county_name'].map(county_name_to_id_map).astype('Int64')

# Cualquier remanente a UNKNOWN (12)
weather_station1.loc[weather_station1['county_name'].isna(), 'county'] = 12
weather_station1['county'] = weather_station1['county'].astype('Int64')

# Limpieza
weather_station1.drop(columns=['county_name_norm'], inplace=True)

# Chequeo rápido
print(weather_station1[['county_name','longitude','latitude','county']].head(25))
print("\nFaltantes aún:", weather_station1['county_name'].isna().sum())


In [ ]:
#weather_station1["county"].value_counts()
#aplicando de esta forma no esta el 12 que es "UNKNOWN"
#supongo que aplica y funciona bien

In [ ]:
# 0 nulos
# 📊 Dataset Shape: (12480, 9)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id']
# 📝 Columnas categóricas: ['prediction_datetime']
# fecha inicio 2023-05-28
#quick_info(test)
#test.head()
#test.describe(include="all")

In [ ]:
# 0 nulos # como identifico a este cliente ?
# 📊 Dataset Shape: (262, 7)
# 🔢 Columnas numéricas: ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'data_block_id']
# 📝 Columnas categóricas: ['date']
#quick_info(client_t)
#client_t.head()
#client_t.describe(include="all")

In [ ]:
# -10 el minimo como el de train , ver que hacer con eso
# 📊 Dataset Shape: (96, 4)
# 🔢 Columnas numéricas: ['euros_per_mwh', 'data_block_id']
# 📝 Columnas categóricas: ['forecast_date', 'origin_date']
#fecha inicio 2023-05-26
#quick_info(electricity_prices_t)
#electricity_prices_t.head()
#electricity_prices_t.describe(include="all")

In [ ]:
# 📊 Dataset Shape: (21504, 18)
# 🔢 Columnas numéricas: ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
#                        'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'data_block_id',
#                        'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
# 📝 Columnas categóricas: ['origin_datetime', 'forecast_datetime']
# fecha inicio 2023-05-27
# quick_info(forecast_weather_t)
# forecast_weather_t.head()
#forecast_weather_t.describe(include="all")

In [ ]:
# 📊 Dataset Shape: (4, 5)
# 🔢 Columnas numéricas: ['lowest_price_per_mwh', 'highest_price_per_mwh', 'data_block_id']
# 📝 Columnas categóricas: ['forecast_date', 'origin_date']
# fecha inicio 2023-5-26
# quick_info(gas_prices_t)
# gas_prices_t.head()
# #gas_prices_t.describe(include="all")

In [ ]:
# 📊 Dataset Shape: (12576, 9)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id']
# 📝 Columnas categóricas: ['datetime']
# fecha inicio 2023-05-26
# quick_info(revealed_targets)
#revealed_targets.head()
#revealed_targets.describe(include="all")

In [ ]:
# lo mejor seria ver como ir combinando todo en 1 dataset ,hago copia y pongo los que estan listos para usar

train1 = train.dropna()# train                 
client1 = client.copy() # client               
electricity_prices1 = electricity_prices.copy() # electricity_prices  
forecast_weather1 = forecast_weather.dropna() # forecast_weather    
gas_prices1 = gas_prices.copy() # gas_prices                        
historical_weather1 = historical_weather.copy() # historical_weather    
weather_station2 = weather_station1.copy()      

test1 = test.copy() # test                 
client_t = client_t.copy() # client_t              
electricity_prices_t1 = electricity_prices_t.copy() # electricity_prices_t  
forecast_weather_t1 = forecast_weather_t.copy() # forecast_weather_t    
gas_prices_t1 = gas_prices_t.copy() # gas_prices_t          
revealed_targets1 = revealed_targets.copy() # revealed_targets    



In [ ]:
#train1.head()

In [ ]:
#quick_info(train1)

# 📊 Dataset Shape: (2017824, 9)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id']
# 📝 Columnas categóricas: ['datetime']
# ❌ Valores nulos por columna:
# Series([], dtype: int64)

In [ ]:
#gas_prices1.tail()

In [ ]:
#quick_info(gas_prices1)

# 📊 Dataset Shape: (637, 5)
# 🔢 Columnas numéricas: ['lowest_price_per_mwh', 'highest_price_per_mwh', 'data_block_id']
# 📝 Columnas categóricas: ['forecast_date', 'origin_date']
# ❌ Valores nulos por columna:
# Series([], dtype: int64)

In [ ]:
# el forecast de gas es de 24 horas

# Convertir datetime y crear columnas separadas en train1
train1['datetime'] = pd.to_datetime(train1['datetime'])
train1['hour'] = train1['datetime'].dt.hour
train1['forecast_date'] = train1['datetime']

#
gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
gas_prices1['hour'] = gas_prices1['forecast_date'].dt.hour
gas_prices1 = gas_prices1.rename(columns={'origin_date': 'gas_origin_date'})




In [ ]:
#train1.tail()

In [ ]:
#gas_prices1.tail(10)

In [ ]:
# Expandir gas_prices1 a nivel horario - solo columnas necesarias
gas_hourly = []
for _, row in gas_prices1.iterrows():
    for hour in range(24):
        new_row = {
            'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
            'lowest_price_per_mwh': row['lowest_price_per_mwh'],
            'highest_price_per_mwh': row['highest_price_per_mwh'],
            'gas_origin_date': row['gas_origin_date']
        }
        gas_hourly.append(new_row)

gas_prices1_hourly = pd.DataFrame(gas_hourly)

# Merge train1 con gas_prices1_hourly
train2 = pd.merge(train1, gas_prices1_hourly, on='forecast_date', how='left')

In [ ]:
#train2.tail()

In [ ]:
#quick_info(train2)

# 📊 Dataset Shape: (2017824, 14)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id', 'hour', 'lowest_price_per_mwh', 'highest_price_per_mwh']
# 📝 Columnas categóricas: []
# ❌ Valores nulos por columna:
# lowest_price_per_mwh     3120
# highest_price_per_mwh    3120
# gas_origin_date          3120
# dtype: int64

In [ ]:
train2['lowest_price_per_mwh'] = train2['lowest_price_per_mwh'].fillna(method='ffill')
train2['highest_price_per_mwh'] = train2['highest_price_per_mwh'].fillna(method='ffill')

# Eliminar columna gas_origin_date
train2 = train2.drop(columns=['gas_origin_date'])

In [ ]:
#quick_info(train2)

In [ ]:
# train2.tail()
# 📊 Dataset Shape: (2017824, 16)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id', 'hour', 'minute', 'second', 'lowest_price_per_mwh', 'highest_price_per_mwh']
# 📝 Columnas categóricas: []
# ❌ Valores nulos por columna:


In [ ]:
#electricity_prices1.tail()

In [ ]:
#quick_info(electricity_prices1)

# 📊 Dataset Shape: (15286, 4)
# 🔢 Columnas numéricas: ['euros_per_mwh', 'data_block_id']
# 📝 Columnas categóricas: ['forecast_date', 'origin_date']
# ❌ Valores nulos por columna:
# Series([], dtype: int64)

In [ ]:
#electricity_prices1.head()

In [ ]:
# Preparar electricity_prices1
electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])

# Extraer componentes de tiempo
electricity_prices1['hour'] = electricity_prices1['forecast_date'].dt.hour


# Merge
elec_columns = ['forecast_date', 'euros_per_mwh']
train3 = pd.merge(train2, electricity_prices1[elec_columns], on='forecast_date', how='left')

In [ ]:
train3['euros_per_mwh'] = train3['euros_per_mwh'].fillna(method='ffill')


In [ ]:
#train3.head()

In [ ]:
#quick_info(train3)

# 📊 Dataset Shape: (2017824, 18)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id', 'hour', 'minute', 'second', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh']
# 📝 Columnas categóricas: []
# ❌ Valores nulos por columna:


In [ ]:
#train3.tail()

In [ ]:
#client1.tail(5)

In [ ]:
quick_info(client1)

# 📊 Dataset Shape: (41919, 7)
# 🔢 Columnas numéricas: ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'data_block_id']
# 📝 Columnas categóricas: ['date']
# ❌ Valores nulos por columna:
# Series([], dtype: int64)

In [ ]:
#train3.shape

In [ ]:
#client1.shape

In [ ]:
# Preparar client1
client1['date'] = pd.to_datetime(client1['date']).dt.date

# Merge con client1 usando datetime de train3
train4 = train3.merge(client1.drop('data_block_id', axis=1),
                      left_on=['product_type', 'county', 'is_business', train3['datetime'].dt.date],
                      right_on=['product_type', 'county', 'is_business', 'date'],
                      how='left')

# Limpiar
train4 = train4.drop('date', axis=1)
train4 = train4.rename(columns={'data_block_id_x': 'data_block_id'})

In [ ]:
#train4.head()

In [ ]:
#quick_info(train4)


# 📊 Dataset Shape: (2017824, 20)
# 🔢 Columnas numéricas: ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id', 'hour', 'minute', 'second', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 'eic_count', 'installed_capacity']
# 📝 Columnas categóricas: []
# ❌ Valores nulos por columna:
# eic_count                6240
# installed_capacity       6240
# dtype: int64


#falta corregir esto todavia

In [ ]:
# decido completar asi los ultimos 2 dias que faltan
# Verificar el problema
print("NaN en datos de clientes:")
print(f"eic_count: {train4['eic_count'].isna().sum():,}")
print(f"installed_capacity: {train4['installed_capacity'].isna().sum():,}")

# Ver las fechas con problemas
nan_dates = train4[train4['eic_count'].isna()]['datetime'].dt.date.unique()
print(f"Fechas con NaN: {sorted(nan_dates)}")

# Ver último día con datos
last_valid_date = train4[train4['eic_count'].notna()]['datetime'].dt.date.max()
print(f"Último día con datos: {last_valid_date}")

# Forward fill por grupo (county + is_business + product_type)
# Esto mantiene los últimos valores conocidos para cada combinación
print("\nCompletando con forward fill por grupo...")

# Ordenar por datetime para asegurar orden correcto
train4 = train4.sort_values(['county', 'is_business', 'product_type', 'datetime'])

# Forward fill por grupo
train4['eic_count'] = train4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
train4['installed_capacity'] = train4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')

# Verificar resultado
print(f"\nDespués del forward fill:")
print(f"eic_count NaN: {train4['eic_count'].isna().sum()}")
print(f"installed_capacity NaN: {train4['installed_capacity'].isna().sum()}")

# Si aún quedan NaN, son combinaciones nuevas - usar 0 o promedio
remaining_nan = train4['eic_count'].isna().sum()
if remaining_nan > 0:
    print(f"\n{remaining_nan} NaN restantes (combinaciones nuevas)")
    print("Opciones:")
    print("1. Rellenar con 0")
    print("2. Rellenar with promedio del county")
    
    # Opción recomendada: promedio por county
    train4['eic_count'] = train4['eic_count'].fillna(
        train4.groupby('county')['eic_count'].transform('mean')
    )
    train4['installed_capacity'] = train4['installed_capacity'].fillna(
        train4.groupby('county')['installed_capacity'].transform('mean')
    )
    
    print(f"NaN finales - eic_count: {train4['eic_count'].isna().sum()}")
    print(f"NaN finales - installed_capacity: {train4['installed_capacity'].isna().sum()}")

print(f"\nDataset train4: {train4.shape[0]:,} filas, {train4.shape[1]} columnas")

In [ ]:
#train4.describe()

In [ ]:
#train4.tail()

In [ ]:
#forecast_weather1.head()

In [ ]:
weather_station3 = weather_station2.copy()
weather_station3.head()

In [ ]:
# Merge forecast_weather1 con weather_station1 por coordenadas
forecast_weather1 = forecast_weather1.merge(weather_station3[['latitude', 'longitude', 'county', 'county_name']], 
                                          on=['latitude', 'longitude'], 
                                          how='left')

print(f"Filas con county asignado: {forecast_weather1['county'].notna().sum()}")
print(f"Total filas: {len(forecast_weather1)}")

In [ ]:
#forecast_weather1.head()

In [ ]:
#forecast_weather1.describe()

In [ ]:
#quick_info(forecast_weather1)

# 📊 Dataset Shape: (3424510, 20)
# 🔢 Columnas numéricas: ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'data_block_id', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation', 'county']
# 📝 Columnas categóricas: ['origin_datetime', 'forecast_datetime', 'county_name']
# ❌ Valores nulos por columna:
# county         2140318
# county_name    2140318
# dtype: int64

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Identificar filas sin county asignado
missing_county = forecast_weather1['county'].isna()
print(f"Filas sin county: {missing_county.sum()}")

if missing_county.sum() > 0:
   # Datos conocidos (con county)
   known = weather_station1[['latitude', 'longitude', 'county', 'county_name']].copy()
   
   # Datos faltantes
   missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
   
   # K-NN con distancia euclidiana (para coordenadas pequeñas como Estonia)
   nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
   nbrs.fit(known[['latitude', 'longitude']].values)
   
   # Encontrar vecino más cercano
   distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
   
   # Asignar county y county_name del vecino más cercano
   nearest_counties = known.iloc[indices.flatten()]
   
   forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
   forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values

print(f"Filas con county después del K-NN: {forecast_weather1['county'].notna().sum()}")

In [ ]:
#quick_info(forecast_weather1)

In [ ]:
#forecast_weather1.tail()
# unir por latitu/longitude y forecast_datetime# esto por ahora no
# sacar county_name
# sacar origin_datetime
# renombrar las variables de forecast con una f adelante (excepto county/hours_ahead/latitude/longitude)
# eliminar origin_datetime
# agregar horas/minutos/segundos y dejar 1 columna con año/mes/dia cuando transforme forecast_datetime

In [ ]:
forecast_weather2 = forecast_weather1.copy()

# Eliminar columnas innecesarias
forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)


# Convertir forecast_datetime a datetime
forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])

# Crear columnas temporales
forecast_weather2['forecast_date'] = forecast_weather2['forecast_datetime'].dt.date
forecast_weather2['hour'] = forecast_weather2['forecast_datetime'].dt.hour

# Renombrar variables con 'f' adelante (excepto las especificadas)
rename_dict = {}
for col in forecast_weather2.columns:
   if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                  'forecast_datetime', 'forecast_date', 'hour']:
       rename_dict[col] = 'f' + col

forecast_weather2 = forecast_weather2.rename(columns=rename_dict)

print("Columnas después de transformaciones:")
print(forecast_weather2.columns.tolist())

forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead":"weather_forecast_hour"})

In [ ]:
#forecast_weather2.tail()

In [ ]:
#historical_weather1.tail()
#agregarle horas/minutos/segundos y cambiar datetime

In [ ]:
#quick_info(historical_weather1)

In [ ]:
historical_weather1['datetime'] = pd.to_datetime(historical_weather1['datetime'])

# Renombrar y merge
forecast_weather2 = forecast_weather2.rename(columns={'data_block_id': 'f_data_block_id'})

merged_weather = pd.merge(
    forecast_weather2,
    historical_weather1,
    left_on=['latitude', 'longitude', 'forecast_datetime'],
    right_on=['latitude', 'longitude', 'datetime'],
    how='inner'
)


In [ ]:
#merged_weather.head()

In [ ]:
#merged_weather.describe()

In [ ]:
#quick_info(merged_weather)

In [ ]:
#train4.head()


In [ ]:
# Optimizar tipos de datos
train4['county'] = train4['county'].astype('uint8')  # 0-15
train4['is_business'] = train4['is_business'].astype('uint8')  # 0-1
train4['product_type'] = train4['product_type'].astype('uint8')  # 0-3
train4['is_consumption'] = train4['is_consumption'].astype('uint8')  # 0-1
train4['hour'] = train4['hour'].astype('uint8')  # 0-23

# IDs optimizados
train4['data_block_id'] = train4['data_block_id'].astype('uint16')  # 0-700
train4['row_id'] = train4['row_id'].astype('uint32')  # 0-3M
train4['prediction_unit_id'] = train4['prediction_unit_id'].astype('uint8')  # 0-68

# Flotantes a float32 (suficiente precision para los rangos dados)
train4['target'] = train4['target'].astype('float32')
train4['lowest_price_per_mwh'] = train4['lowest_price_per_mwh'].astype('float32')
train4['highest_price_per_mwh'] = train4['highest_price_per_mwh'].astype('float32')
train4['euros_per_mwh'] = train4['euros_per_mwh'].astype('float32')
train4['eic_count'] = train4['eic_count'].astype('float32')
train4['installed_capacity'] = train4['installed_capacity'].astype('float32')

print("Optimización completada")
print(train4.info(memory_usage='deep'))

In [ ]:
# Eliminar columnas innecesarias
merged_weather = merged_weather.drop('f_data_block_id', axis=1)

# Coordenadas - float32 suficiente para la precisión geográfica
merged_weather['latitude'] = merged_weather['latitude'].astype('float32')  # 57.6-59.7
merged_weather['longitude'] = merged_weather['longitude'].astype('float32')  # 21.7-28.2

# Enteros pequeños
merged_weather['weather_forecast_hour'] = merged_weather['weather_forecast_hour'].astype('uint8')  # 1-48
merged_weather['county'] = merged_weather['county'].astype('uint8')  # 0-15
merged_weather['hour'] = merged_weather['hour'].astype('uint8')  # 0-23

# IDs
merged_weather['data_block_id'] = merged_weather['data_block_id'].astype('uint16')  # 1-637

# Variables meteorológicas - float32 es suficiente
weather_float_cols = ['ftemperature', 'fdewpoint', 'fcloudcover_high', 'fcloudcover_low', 
                     'fcloudcover_mid', 'fcloudcover_total', 'f10_metre_u_wind_component', 
                     'f10_metre_v_wind_component', 'fdirect_solar_radiation', 
                     'fsurface_solar_radiation_downwards', 'fsnowfall', 'ftotal_precipitation',
                     'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure',
                     'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high',
                     'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 
                     'direct_solar_radiation', 'diffuse_radiation']

for col in weather_float_cols:
    merged_weather[col] = merged_weather[col].astype('float32')

print("Optimización de merged_weather completada")
print(merged_weather.info(memory_usage='deep'))

In [ ]:
#print(train4.info(memory_usage='deep'))

#ANTES
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 2017824 entries, 0 to 2017823
# Data columns (total 22 columns):
#  #   Column                 Dtype         
# ---  ------                 -----         
#  0   county                 int64         
#  1   is_business            int64         
#  2   product_type           int64         
#  3   target                 float64       
#  4   is_consumption         int64         
#  5   datetime               datetime64[ns]
#  6   data_block_id          int64         
#  7   row_id                 int64         
#  8   prediction_unit_id     int64         
#  9   hour                   int32         
#  10  minute                 int32         
#  11  second                 int32         
#  12  forecast_date          datetime64[ns]
#  13  lowest_price_per_mwh   float64       
#  14  highest_price_per_mwh  float64       
#  15  gas_origin_date        datetime64[ns]
#  16  euros_per_mwh          float64       
#  17  elec_origin_date       datetime64[ns]
#  18  eic_count              float64       
#  19  installed_capacity     float64       
#  20  elec_forecast_hour     int64         
#  21  gas_forecast_hour      int64         
# dtypes: datetime64[ns](4), float64(6), int32(3), int64(9)
# memory usage: 315.6 MB
# None

In [ ]:
#print(merged_weather.info(memory_usage='deep'))

# Antes
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 3418242 entries, 0 to 3418241
# Data columns (total 38 columns):
#  #   Column                              Dtype         
# ---  ------                              -----         
#  0   latitude                            float64     # 2 digitos y 1 decimal  
#  1   longitude                           float64     # 2 digitos y 1 decimal   
#  2   weather_forecast_hour               int64       # 0 a 4 digitos 
#  3   ftemperature                        float64     #  
#  4   fdewpoint                           float64       
#  5   fcloudcover_high                    float64       
#  6   fcloudcover_low                     float64       
#  7   fcloudcover_mid                     float64       
#  8   fcloudcover_total                   float64       
#  9   f10_metre_u_wind_component          float64       
#  10  f10_metre_v_wind_component          float64       
#  11  f_data_block_id                     int64         
#  12  forecast_datetime                   datetime64[ns]
#  13  fdirect_solar_radiation             float64       
#  14  fsurface_solar_radiation_downwards  float64       
#  15  fsnowfall                           float64       
#  16  ftotal_precipitation                float64       
#  17  county                              Int64         
#  18  forecast_date                       object        
#  19  hour                                int32         
#  20  minute                              int32         
#  21  second                              int32         
#  22  datetime                            datetime64[ns]
#  23  temperature                         float64       
#  24  dewpoint                            float64       
#  25  rain                                float64       
#  26  snowfall                            float64       
#  27  surface_pressure                    float64       
#  28  cloudcover_total                    int64         
#  29  cloudcover_low                      int64         
#  30  cloudcover_mid                      int64         
#  31  cloudcover_high                     int64         
#  32  windspeed_10m                       float64       
#  33  winddirection_10m                   int64         
#  34  shortwave_radiation                 float64       
#  35  direct_solar_radiation              float64       
#  36  diffuse_radiation                   float64       
#  37  data_block_id                       float64       
# dtypes: Int64(1), datetime64[ns](2), float64(24), int32(3), int64(7), object(1)
# memory usage: 1.0 GB
# None

In [ ]:
del train3, train2

In [ ]:
merged_weather = merged_weather.drop('forecast_datetime', axis=1)

In [ ]:
#train4.head()

In [ ]:
#merged_weather.head()

In [ ]:
del train, client, electricity_prices, forecast_weather, gas_prices, historical_weather, weather_station

In [ ]:
#merge por county y datetime
def process_weather_chunks_corrected(start_chunk, end_chunk):
    periods = [
        ('2021-09-01', '2021-10-01'), ('2021-10-01', '2021-11-01'), ('2021-11-01', '2021-12-01'), ('2021-12-01', '2022-01-01'),
        ('2022-01-01', '2022-02-01'), ('2022-02-01', '2022-03-01'), ('2022-03-01', '2022-04-01'), ('2022-04-01', '2022-05-01'),
        ('2022-05-01', '2022-06-01'), ('2022-06-01', '2022-07-01'), ('2022-07-01', '2022-08-01'), ('2022-08-01', '2022-09-01'),
        ('2022-09-01', '2022-10-01'), ('2022-10-01', '2022-11-01'), ('2022-11-01', '2022-12-01'), ('2022-12-01', '2023-01-01'),
        ('2023-01-01', '2023-02-01'), ('2023-02-01', '2023-03-01'), ('2023-03-01', '2023-04-01'), ('2023-04-01', '2023-05-01'),
        ('2023-05-01', '2023-05-31')
    ]
    
    for i in range(start_chunk, end_chunk):
        start_date, end_date = periods[i]
        print(f"Procesando período {i+1}: {start_date} a {end_date}")
        
        # Filtrar por período
        mask_train = (train4['datetime'] >= start_date) & (train4['datetime'] < end_date)
        train4_chunk = train4[mask_train].copy()
        
        mask_weather = (merged_weather['datetime'] >= start_date) & (merged_weather['datetime'] < end_date)
        weather_chunk = merged_weather[mask_weather].copy()
        
        # Merge CORREGIDO - por county y datetime
        chunk_result = pd.merge(train4_chunk, weather_chunk, 
                               on=['county', 'datetime'], 
                               how='left')
        
        chunk_result.to_parquet(f'train5_corrected_chunk_{i+1}.parquet', index=False)
        print(f"Chunk {i+1} guardado: {chunk_result.shape}")
        
        # Liberar memoria
        del train4_chunk, weather_chunk, chunk_result

# Probar con un chunk pequeño primero
process_weather_chunks_corrected(0, 1)

In [ ]:
process_weather_chunks_corrected(0, 3)


In [ ]:
process_weather_chunks_corrected(3, 9)  


In [ ]:
process_weather_chunks_corrected(9, 15)  

In [ ]:
process_weather_chunks_corrected(15, 21)  

In [ ]:
# Concatenar de a grupos para evitar saturar memoria
def concat_chunks_by_groups(group_size=1):
    group_files = []
    
    # Crear grupos intermedios
    for group_start in range(1, 22, group_size):
        group_end = min(group_start + group_size, 22)
        print(f"Procesando grupo: chunks {group_start} a {group_end-1}")
        
        # Leer chunks del grupo
        group_chunks = []
        for i in range(group_start, group_end):
            try:
                chunk = pd.read_parquet(f'train5_corrected_chunk_{i}.parquet')  # CORREGIDO
                group_chunks.append(chunk)
            except FileNotFoundError:
                print(f"Archivo train5_corrected_chunk_{i}.parquet no encontrado")
        
        # Concatenar grupo y guardar
        if group_chunks:
            group_df = pd.concat(group_chunks, ignore_index=True)
            group_filename = f'train5_group_{group_start}_{group_end-1}.parquet'
            group_df.to_parquet(group_filename, index=False)
            group_files.append(group_filename)
            print(f"Grupo guardado: {group_filename} - Shape: {group_df.shape}")
            
            # Liberar memoria
            del group_chunks, group_df
    
    # Ahora concatenar los grupos (serán muchos menos archivos)
    print("Concatenando grupos finales...")
    final_chunks = []
    for group_file in group_files:
        group_data = pd.read_parquet(group_file)
        final_chunks.append(group_data)
    
    train5 = pd.concat(final_chunks, ignore_index=True)
    return train5

# Ejecutar
train5 = concat_chunks_by_groups(1)
print(f"Dataset final: {train5.shape}")

In [ ]:
#train5.head()

In [ ]:
#quick_info(train5)

In [ ]:
import gc

# 1. Identificar columnas a eliminar y renombrar SIN cargar datos
sample = train5.head(1000)  # Solo una muestra
cols_to_drop = [col for col in sample.columns if col.endswith('_y')]
rename_dict = {col: col[:-2] for col in sample.columns if col.endswith('_x')}
print(f"Eliminando {len(cols_to_drop)} columnas, renombrando {len(rename_dict)}")

# 2. Procesar por chunks de 1M filas
chunk_size = 1_000_000
chunks_processed = []

total_rows = len(train5)
n_chunks = (total_rows // chunk_size) + 1

for i in range(n_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, total_rows)
    
    print(f"Procesando chunk {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
    
    # Procesar chunk
    chunk = train5.iloc[start_idx:end_idx].copy()
    
    # Filtrar county != 12
    chunk = chunk[chunk['county'] != 12]
    
    if len(chunk) > 0:  # Solo si quedan filas
        # Eliminar columnas _y
        chunk = chunk.drop(columns=cols_to_drop)
        
        # Renombrar _x
        chunk.rename(columns=rename_dict, inplace=True)
        
        chunks_processed.append(chunk)
    
    # Limpiar memoria
    del chunk
    gc.collect()
    
    print(f"Memoria liberada, chunks guardados: {len(chunks_processed)}")

# 3. Concatenar chunks finales
print("Concatenando chunks finales...")
train5_clean = pd.concat(chunks_processed, ignore_index=True)

# 4. Limpiar
del chunks_processed, train5
gc.collect()

print(f"Dataset final: {train5_clean.shape[0]:,} filas y {train5_clean.shape[1]} columnas")

# Renombrar para usar el mismo nombre
train5 = train5_clean
del train5_clean
gc.collect()

In [ ]:
train5.head()

In [ ]:
train5.isnull().sum()

In [ ]:
# Orden específico que solicitaste
main_cols = [
    'row_id', 'target', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
    'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
    'longitude', 'lowest_price_per_mwh', 
    'highest_price_per_mwh', 'euros_per_mwh', 'eic_count', 'installed_capacity'
]

# Pares de clima: pronóstico (f) vs histórico - lado a lado para comparar
weather_pairs = [
    # Temperatura
    'ftemperature', 'temperature',
    
    # Punto de rocío  
    'fdewpoint', 'dewpoint',
    
    # Cobertura de nubes
    'fcloudcover_high', 'cloudcover_high',
    'fcloudcover_low', 'cloudcover_low', 
    'fcloudcover_mid', 'cloudcover_mid',
    'fcloudcover_total', 'cloudcover_total',
    
    # Viento
    'f10_metre_u_wind_component', 'windspeed_10m',
    'f10_metre_v_wind_component', 'winddirection_10m',
    
    # Radiación solar
    'fdirect_solar_radiation', 'direct_solar_radiation',
    'fsurface_solar_radiation_downwards', 'shortwave_radiation',
    'diffuse_radiation',  # solo histórico
    
    # Precipitación
    'fsnowfall', 'snowfall',
    'ftotal_precipitation', 'rain',
    
    # Otras variables climáticas
    'surface_pressure',  # solo histórico
    'weather_forecast_hour'
]

# Construir orden final
final_order = main_cols.copy()

# Agregar solo las columnas que existen
for col in weather_pairs:
    if col in train5.columns:
        final_order.append(col)

# Agregar cualquier columna restante
remaining_cols = [col for col in train5.columns if col not in final_order]
final_order.extend(remaining_cols)

print(f"Reordenando {len(final_order)} columnas...")
print("Pares clima encontrados:")
weather_cols_found = [col for col in weather_pairs if col in train5.columns]
for i in range(0, len(weather_cols_found), 2):
    pair = weather_cols_found[i:i+2]
    print(f"  {' vs '.join(pair)}")

# Reordenar
train5 = train5[final_order]

print("¡Columnas reordenadas!")
print(f"Primeras 10: {list(train5.columns[:10])}")
print(f"Clima inicia en posición: {final_order.index(weather_cols_found[0]) if weather_cols_found else 'N/A'}")

In [ ]:
train5.tail(5)

In [ ]:
# hacer scatterplot con 
# train3.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) el alpha para ver con mas claridad las zonas mas poblada
# podria agregar colores para diferencia temperatura o exposicion a rayos solares o nubes y poder encarar distintos el dataset
# sino funciona probar %matplotlib inline
# train3.hist(bins=50, figsize=(20,15)) ver tmb los histograma de los datos cuando este todo junto# cambiar los bins a valores mas chicos
# plt.show() agregar si hace falta
# corr_matrix = train3.corr()
# corr_matrix["target"].sort_values(ascending=False) ver si aplica pero creo que no , recordad probar en columnas numericas si hay otra relacion
# ejemplo crear columna con raiz/potencia/log y ver como apartan esos valores en correlacion
# o usar scatter_matrix() de pandas

In [ ]:
from collections import defaultdict

# Procesar por chunks de 1M filas
chunk_size = 1_000_000
total_rows = len(train5)
n_chunks = (total_rows // chunk_size) + 1

# Variables para acumular resultados
missing_dates_count = defaultdict(int)
missing_by_county = defaultdict(int)
first_missing = None
last_complete = None
total_missing = 0

print(f"Procesando {total_rows:,} filas en {n_chunks} chunks...")

weather_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']

for i in range(n_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, total_rows)
    
    print(f"Chunk {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
    
    # Procesar chunk
    chunk = train5.iloc[start_idx:end_idx][['datetime', 'county'] + weather_cols]
    
    # Encontrar filas con NaN en clima
    missing_mask = chunk[weather_cols].isna().any(axis=1)
    missing_chunk = chunk[missing_mask]
    
    if len(missing_chunk) > 0:
        # Acumular conteos por fecha
        dates = missing_chunk['datetime'].dt.date.value_counts()
        for date, count in dates.items():
            missing_dates_count[date] += count
        
        # Acumular por county
        counties = missing_chunk['county'].value_counts()
        for county, count in counties.items():
            missing_by_county[county] += count
        
        # Tracking de primera fecha faltante
        chunk_first_missing = missing_chunk['datetime'].min()
        if first_missing is None or chunk_first_missing < first_missing:
            first_missing = chunk_first_missing
        
        total_missing += len(missing_chunk)
    
    # Tracking de última fecha completa
    complete_chunk = chunk[chunk['latitude'].notna()]
    if len(complete_chunk) > 0:
        chunk_last_complete = complete_chunk['datetime'].max()
        if last_complete is None or chunk_last_complete > last_complete:
            last_complete = chunk_last_complete
    
    # Limpiar memoria
    del chunk, missing_chunk
    gc.collect()
    
    if i % 5 == 0:  # cada 5 chunks
        print(f"  Faltantes encontrados hasta ahora: {total_missing:,}")

# Mostrar resultados
print(f"\n=== RESULTADOS ===")
print(f"Total filas con clima faltante: {total_missing:,}")
print(f"Total fechas afectadas: {len(missing_dates_count)}")

if missing_dates_count:
    # Convertir a Series para ordenar
    missing_dates_series = pd.Series(missing_dates_count).sort_index()
    
    print(f"\nPrimeras 10 fechas con más faltantes:")
    print(missing_dates_series.head(10))
    
    print(f"\nRango de fechas problemáticas:")
    print(f"Desde: {missing_dates_series.index.min()}")
    print(f"Hasta: {missing_dates_series.index.max()}")
    
    print(f"\nTop 5 counties con más faltantes:")
    county_series = pd.Series(missing_by_county).sort_values(ascending=False)
    print(county_series.head())
    
    print(f"\nTimeline:")
    print(f"Última fecha completa: {last_complete}")
    print(f"Primera fecha faltante: {first_missing}")
    if last_complete and first_missing:
        print(f"Gap de tiempo: {first_missing - last_complete}")
else:
    print("No se encontraron datos de clima faltantes!")

gc.collect()

In [ ]:
# hago drop de los faltantes

# Eliminar filas con datos de clima faltantes
print(f"Dataset original: {len(train5):,} filas")

# Drop filas con NaN en clima
weather_key_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']
train5 = train5.dropna(subset=weather_key_cols)

print(f"Dataset limpio: {len(train5):,} filas")
print(f"Filas eliminadas: {32_965_048 - len(train5):,}")

# Verificar que no quedan NaN en clima
remaining_weather_nan = train5[weather_key_cols].isna().sum().sum()
print(f"NaN restantes en clima: {remaining_weather_nan}")

# Verificar fechas disponibles para el período de predicción
test_period = train5[
    (train5['datetime'].dt.date >= pd.to_datetime('2023-05-20').date()) &
    (train5['datetime'].dt.date <= pd.to_datetime('2023-05-31').date())
]['datetime'].dt.date.unique()

print(f"\nFechas disponibles cerca del período de test:")
print(sorted(test_period))

In [ ]:
train5.isnull().sum()

In [ ]:
######################################

In [ ]:
a

In [ ]:
#Inicio de guia para hacer Pipeline
#Voy pegando los codigos de lo que fui aplicando asi podes usarlo de guia
#quiero poder aplicarlo a Train y a Test ,
#si hay diferencias con test y queres hacer Un Pipeline para Train y otro Pipeline para Test para que sea mas simple , me da igual
#esto termina teniendo 33kk de filas en el final ,asi que tene en cuenta opciones que vayan haciendo de a 1kk para no saturar la memoria , o ir borrando

#fui poniendo los pasos que fui haciendo , obviamente hay que optimizarlo y hay cosas que no son necesarias
#me parece que para train podria ser en 3 partes , primero preparar todo hasta antes del ultimo merge , luego el merge y tercero encargarse de los problemas del merge
#se que para trabajar luego deberia haber otro pipeline para usar standarcaler y otras cosas mas , pero quiero dejarlo hasta aca para ver por mi mismo 
# de que forma continuar




#1er paso el weather to county que utilizo en train y luego voy a utilizar en test

import unicodedata
from sklearn.neighbors import NearestNeighbors

# === 1) Diccionarios canonicos ===
county_id_to_name_map = {
    0: "HARJUMAA", 1: "HIIUMAA", 2: "IDA-VIRUMAA", 3: "JÄRVAMAA",
    4: "JÕGEVAMAA", 5: "LÄÄNE-VIRUMAA", 6: "LÄÄNEMAA", 7: "PÄRNUMAA",
    8: "PÕLVAMAA", 9: "RAPLAMAA", 10: "SAAREMAA", 11: "TARTUMAA",
    12: "UNKNOWN", 13: "VALGAMAA", 14: "VILJANDIMAA", 15: "VÕRUMAA"
}
county_name_to_id_map = {v: k for k, v in county_id_to_name_map.items()}

# Para normalizar entradas (sin/ con acentos, "County"/"maakond", etc.)
synonyms_to_canonical = {
    # clave "base" (sin acentos/variantes) -> canon con acentos
    "HARJU": "HARJUMAA",
    "HIIU": "HIIUMAA",
    "IDA-VIRU": "IDA-VIRUMAA",
    "JARVA": "JÄRVAMAA", "JÄRVA": "JÄRVAMAA",
    "JOGEVA": "JÕGEVAMAA", "JÕGEVA": "JÕGEVAMAA",
    "LAANE-VIRU": "LÄÄNE-VIRUMAA", "LÄÄNE-VIRU": "LÄÄNE-VIRUMAA",
    "LAANE": "LÄÄNEMAA", "LÄÄNE": "LÄÄNEMAA",
    "PARNU": "PÄRNUMAA", "PÄRNU": "PÄRNUMAA",
    "POLVA": "PÕLVAMAA", "PÕLVA": "PÕLVAMAA",
    "RAPLA": "RAPLAMAA",
    "SAARE": "SAAREMAA",
    "TARTU": "TARTUMAA",
    "VALGA": "VALGAMAA",
    "VILJANDI": "VILJANDIMAA",
    "VORU": "VÕRUMAA", "VÕRU": "VÕRUMAA",

    # versiones ya canónicas
    "HARJUMAA": "HARJUMAA",
    "HIIUMAA": "HIIUMAA",
    "IDA-VIRUMAA": "IDA-VIRUMAA",
    "JÄRVAMAA": "JÄRVAMAA",
    "JÕGEVAMAA": "JÕGEVAMAA",
    "LÄÄNE-VIRUMAA": "LÄÄNE-VIRUMAA",
    "LÄÄNEMAA": "LÄÄNEMAA",
    "PÄRNUMAA": "PÄRNUMAA",
    "PÕLVAMAA": "PÕLVAMAA",
    "RAPLAMAA": "RAPLAMAA",
    "SAAREMAA": "SAAREMAA",
    "TARTUMAA": "TARTUMAA",
    "VALGAMAA": "VALGAMAA",
    "VILJANDIMAA": "VILJANDIMAA",
    "VÕRUMAA": "VÕRUMAA",
}

def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))

def normalize_county_name(x):
    if pd.isna(x):
        return np.nan
    s = str(x).upper().strip()
    # eliminar sufijos frecuentes
    for suf in [" COUNTY", " MAAKOND"]:
        if s.endswith(suf):
            s = s[: -len(suf)]
    s = s.replace("  ", " ").replace("–", "-")  # guion raro -> normal
    s_noacc = strip_accents(s)
    s_noacc = s_noacc.replace("  ", " ")
    # prueba de match exacto o por prefijo lógico
    # ej. "HARJU", "HARJU MAAKOND" -> "HARJU"
    token = s_noacc.split()[0]  # primer palabra alcanza en estos casos
    # casos con guion (IDA-VIRU, LAA NE-VIRU, etc.)
    if "-" in s_noacc:
        token = s_noacc  # mantener completo en compuestos

    # intento 1: match directo con cadena completa sin acentos
    if s_noacc in synonyms_to_canonical:
        return synonyms_to_canonical[s_noacc]
    # intento 2: match por token/prefijo conocido
    if token in synonyms_to_canonical:
        return synonyms_to_canonical[token]
    # intento 3: si ya vino canónico exacto (con acentos)
    if s in synonyms_to_canonical:
        return synonyms_to_canonical[s]

    return np.nan  # no lo reconozco

# === 2) Normalizar lo existente sin romper los NaN ===
# (NO convertir NaN a "NAN")
weather_station1['county_name_norm'] = weather_station1['county_name'].apply(normalize_county_name)

# Si tengo 'county' numérico pero falta nombre, lo inferimos del ID
mask = weather_station1['county_name_norm'].isna() & weather_station1['county'].notna()
weather_station1.loc[mask, 'county_name_norm'] = (
    weather_station1.loc[mask, 'county'].astype(int).map(county_id_to_name_map)
)

# === 3) Completar faltantes por k-NN (lat/lon) ===
fcols = ['latitude', 'longitude']
known = weather_station1.dropna(subset=['county_name_norm']).copy()
unknown = weather_station1[weather_station1['county_name_norm'].isna()].copy()

if not unknown.empty and not known.empty:
    # k-NN en (lat, lon) usando Haversine (necesita radianes)
    # ojo: haversine en sklearn requiere [lat, lon] en radianes
    known_rad = np.radians(known[fcols].values)
    unknown_rad = np.radians(unknown[fcols].values)

    nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
    nbrs.fit(known_rad)
    distances, idxs = nbrs.kneighbors(unknown_rad)

    # votación simple de los 3 vecinos (majority vote)
    nn_names = known['county_name_norm'].values
    filled = []
    for neigh_idx in idxs:
        cands = nn_names[neigh_idx]
        # mayoría; en empate, tomar el 1er vecino
        vals, counts = np.unique(cands, return_counts=True)
        filled.append(vals[np.argmax(counts)])

    weather_station1.loc[unknown.index, 'county_name_norm'] = filled

# === 4) Sincronizar columnas finales ===
weather_station1['county_name'] = weather_station1['county_name_norm']
weather_station1['county'] = weather_station1['county_name'].map(county_name_to_id_map).astype('Int64')

# Cualquier remanente a UNKNOWN (12)
weather_station1.loc[weather_station1['county_name'].isna(), 'county'] = 12
weather_station1['county'] = weather_station1['county'].astype('Int64')

# Limpieza
weather_station1.drop(columns=['county_name_norm'], inplace=True)

# Chequeo rápido
print(weather_station1[['county_name','longitude','latitude','county']].head(25))
print("\nFaltantes aún:", weather_station1['county_name'].isna().sum())


#2do paso crear copias antes de empezar a juntar/merge los df , las cuales elimino al final para liberar memoria
# en algunos caso elimino los na con drop , en otros no hace falta respetar como esta aca 
# en test no hay historical porque estoy prediciendo a futuro 
# weather_station2 es asi por lo que hice arriba

train1 = train.dropna()# train                 
client1 = client.copy() # client               
electricity_prices1 = electricity_prices.copy() # electricity_prices  
forecast_weather1 = forecast_weather.dropna() # forecast_weather    
gas_prices1 = gas_prices.copy() # gas_prices                        
historical_weather1 = historical_weather.copy() # historical_weather    
weather_station2 = weather_station1.copy()      

test1 = test.copy() # test                 
client_t = client_t.copy() # client_t              
electricity_prices_t1 = electricity_prices_t.copy() # electricity_prices_t  
forecast_weather_t1 = forecast_weather_t.copy() # forecast_weather_t    
gas_prices_t1 = gas_prices_t.copy() # gas_prices_t          
revealed_targets1 = revealed_targets.copy() # revealed_targets

#3er paso trabajo con train1 y gas_prices1 para hacer el primer merge

# el forecast de gas es de 24 horas

# Convertir datetime y crear columnas separadas en train1
train1['datetime'] = pd.to_datetime(train1['datetime'])
train1['hour'] = train1['datetime'].dt.hour
train1['forecast_date'] = train1['datetime']

#
gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
gas_prices1['hour'] = gas_prices1['forecast_date'].dt.hour
gas_prices1 = gas_prices1.rename(columns={'origin_date': 'gas_origin_date'})

# Expandir gas_prices1 a nivel horario - solo columnas necesarias
gas_hourly = []
for _, row in gas_prices1.iterrows():
    for hour in range(24):
        new_row = {
            'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
            'lowest_price_per_mwh': row['lowest_price_per_mwh'],
            'highest_price_per_mwh': row['highest_price_per_mwh'],
            'gas_origin_date': row['gas_origin_date']
        }
        gas_hourly.append(new_row)

gas_prices1_hourly = pd.DataFrame(gas_hourly)

# Merge train1 con gas_prices1_hourly
train2 = pd.merge(train1, gas_prices1_hourly, on='forecast_date', how='left')
# completo los datos faltantes antes de continuar
train2['lowest_price_per_mwh'] = train2['lowest_price_per_mwh'].fillna(method='ffill')
train2['highest_price_per_mwh'] = train2['highest_price_per_mwh'].fillna(method='ffill')

# Eliminar columna gas_origin_date
train2 = train2.drop(columns=['gas_origin_date'])

# ahora se llama train2 que es train1+gas_prices1
# a "gas_origin_date" despues lo elimino , si lees todo el pipeline y cuando lo corrijas ves que es mejor eliminarlo ahora , hacelo ahora 



#3er paso


# Preparar electricity_prices1 para el merge con train2
electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])

# Extraer componentes de tiempo
electricity_prices1['hour'] = electricity_prices1['forecast_date'].dt.hour


# Merge
elec_columns = ['forecast_date', 'euros_per_mwh']
train3 = pd.merge(train2, electricity_prices1[elec_columns], on='forecast_date', how='left')

# me ocupo de los valores faltantes antes de continuar

train3['euros_per_mwh'] = train3['euros_per_mwh'].fillna(method='ffill')


#4to paso , preparo client1 para unirlo con train3

# Preparar client1
client1['date'] = pd.to_datetime(client1['date']).dt.date

# Merge con client1 usando datetime de train3
train4 = train3.merge(client1.drop('data_block_id', axis=1),
                      left_on=['product_type', 'county', 'is_business', train3['datetime'].dt.date],
                      right_on=['product_type', 'county', 'is_business', 'date'],
                      how='left')

# Limpiar
train4 = train4.drop('date', axis=1)
train4 = train4.rename(columns={'data_block_id_x': 'data_block_id'})

# me ocupo de los valores faltantes antes de continuar
# decido completar asi los ultimos 2 dias que faltan
# Verificar el problema
print("NaN en datos de clientes:")
print(f"eic_count: {train4['eic_count'].isna().sum():,}")
print(f"installed_capacity: {train4['installed_capacity'].isna().sum():,}")

# Ver las fechas con problemas
nan_dates = train4[train4['eic_count'].isna()]['datetime'].dt.date.unique()
print(f"Fechas con NaN: {sorted(nan_dates)}")

# Ver último día con datos
last_valid_date = train4[train4['eic_count'].notna()]['datetime'].dt.date.max()
print(f"Último día con datos: {last_valid_date}")

# Forward fill por grupo (county + is_business + product_type)
# Esto mantiene los últimos valores conocidos para cada combinación
print("\nCompletando con forward fill por grupo...")

# Ordenar por datetime para asegurar orden correcto
train4 = train4.sort_values(['county', 'is_business', 'product_type', 'datetime'])

# Forward fill por grupo
train4['eic_count'] = train4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
train4['installed_capacity'] = train4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')

# Verificar resultado
print(f"\nDespués del forward fill:")
print(f"eic_count NaN: {train4['eic_count'].isna().sum()}")
print(f"installed_capacity NaN: {train4['installed_capacity'].isna().sum()}")

# Si aún quedan NaN, son combinaciones nuevas - usar 0 o promedio
remaining_nan = train4['eic_count'].isna().sum()
if remaining_nan > 0:
    print(f"\n{remaining_nan} NaN restantes (combinaciones nuevas)")
    print("Opciones:")
    print("1. Rellenar con 0")
    print("2. Rellenar with promedio del county")
    
    # Opción recomendada: promedio por county
    train4['eic_count'] = train4['eic_count'].fillna(
        train4.groupby('county')['eic_count'].transform('mean')
    )
    train4['installed_capacity'] = train4['installed_capacity'].fillna(
        train4.groupby('county')['installed_capacity'].transform('mean')
    )
    
    print(f"NaN finales - eic_count: {train4['eic_count'].isna().sum()}")
    print(f"NaN finales - installed_capacity: {train4['installed_capacity'].isna().sum()}")

print(f"\nDataset train4: {train4.shape[0]:,} filas, {train4.shape[1]} columnas")

# 5to paso antes de continuar con train4 sigo con los df relacionados al clima , junto forecast_weather1 con weather station1 ,

# Merge forecast_weather1 con weather_station1 por coordenadas
forecast_weather1 = forecast_weather1.merge(weather_station3[['latitude', 'longitude', 'county', 'county_name']], 
                                          on=['latitude', 'longitude'], 
                                          how='left')

print(f"Filas con county asignado: {forecast_weather1['county'].notna().sum()}")
print(f"Total filas: {len(forecast_weather1)}")

# completo los valores faltantes

from sklearn.neighbors import NearestNeighbors
import numpy as np

# Identificar filas sin county asignado
missing_county = forecast_weather1['county'].isna()
print(f"Filas sin county: {missing_county.sum()}")

if missing_county.sum() > 0:
   # Datos conocidos (con county)
   known = weather_station1[['latitude', 'longitude', 'county', 'county_name']].copy()
   
   # Datos faltantes
   missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
   
   # K-NN con distancia euclidiana (para coordenadas pequeñas como Estonia)
   nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
   nbrs.fit(known[['latitude', 'longitude']].values)
   
   # Encontrar vecino más cercano
   distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
   
   # Asignar county y county_name del vecino más cercano
   nearest_counties = known.iloc[indices.flatten()]
   
   forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
   forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values

print(f"Filas con county después del K-NN: {forecast_weather1['county'].notna().sum()}")

#6to paso trabajo sobre forecast_weather 

forecast_weather2 = forecast_weather1.copy()

# Eliminar columnas innecesarias
forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)


# Convertir forecast_datetime a datetime
forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])

# Crear columnas temporales
forecast_weather2['forecast_date'] = forecast_weather2['forecast_datetime'].dt.date
forecast_weather2['hour'] = forecast_weather2['forecast_datetime'].dt.hour

# Renombrar variables con 'f' adelante (excepto las especificadas)
rename_dict = {}
for col in forecast_weather2.columns:
   if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                  'forecast_datetime', 'forecast_date', 'hour']:
       rename_dict[col] = 'f' + col

forecast_weather2 = forecast_weather2.rename(columns=rename_dict)

print("Columnas después de transformaciones:")
print(forecast_weather2.columns.tolist())

forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead":"weather_forecast_hour"})

#7mo paso preparo y hago merge con historical weather

historical_weather1['datetime'] = pd.to_datetime(historical_weather1['datetime'])

# Renombrar y merge
forecast_weather2 = forecast_weather2.rename(columns={'data_block_id': 'f_data_block_id'})

merged_weather = pd.merge(
    forecast_weather2,
    historical_weather1,
    left_on=['latitude', 'longitude', 'forecast_datetime'],
    right_on=['latitude', 'longitude', 'datetime'],
    how='inner'
)

#8vo paso , optimizo datos por un tema de memoria antes de continuar

# Optimizar tipos de datos
train4['county'] = train4['county'].astype('uint8')  # 0-15
train4['is_business'] = train4['is_business'].astype('uint8')  # 0-1
train4['product_type'] = train4['product_type'].astype('uint8')  # 0-3
train4['is_consumption'] = train4['is_consumption'].astype('uint8')  # 0-1
train4['hour'] = train4['hour'].astype('uint8')  # 0-23

# IDs optimizados
train4['data_block_id'] = train4['data_block_id'].astype('uint16')  # 0-700
train4['row_id'] = train4['row_id'].astype('uint32')  # 0-3M
train4['prediction_unit_id'] = train4['prediction_unit_id'].astype('uint8')  # 0-68

# Flotantes a float32 (suficiente precision para los rangos dados)
train4['target'] = train4['target'].astype('float32')
train4['lowest_price_per_mwh'] = train4['lowest_price_per_mwh'].astype('float32')
train4['highest_price_per_mwh'] = train4['highest_price_per_mwh'].astype('float32')
train4['euros_per_mwh'] = train4['euros_per_mwh'].astype('float32')
train4['eic_count'] = train4['eic_count'].astype('float32')
train4['installed_capacity'] = train4['installed_capacity'].astype('float32')

print("Optimización completada")
print(train4.info(memory_usage='deep'))

# optimizo y preparo para merge con train4 

# Eliminar columnas innecesarias
merged_weather = merged_weather.drop('f_data_block_id', axis=1)

# Coordenadas - float32 suficiente para la precisión geográfica
merged_weather['latitude'] = merged_weather['latitude'].astype('float32')  # 57.6-59.7
merged_weather['longitude'] = merged_weather['longitude'].astype('float32')  # 21.7-28.2

# Enteros pequeños
merged_weather['weather_forecast_hour'] = merged_weather['weather_forecast_hour'].astype('uint8')  # 1-48
merged_weather['county'] = merged_weather['county'].astype('uint8')  # 0-15
merged_weather['hour'] = merged_weather['hour'].astype('uint8')  # 0-23

# IDs
merged_weather['data_block_id'] = merged_weather['data_block_id'].astype('uint16')  # 1-637

# Variables meteorológicas - float32 es suficiente
weather_float_cols = ['ftemperature', 'fdewpoint', 'fcloudcover_high', 'fcloudcover_low', 
                     'fcloudcover_mid', 'fcloudcover_total', 'f10_metre_u_wind_component', 
                     'f10_metre_v_wind_component', 'fdirect_solar_radiation', 
                     'fsurface_solar_radiation_downwards', 'fsnowfall', 'ftotal_precipitation',
                     'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure',
                     'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high',
                     'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 
                     'direct_solar_radiation', 'diffuse_radiation']

for col in weather_float_cols:
    merged_weather[col] = merged_weather[col].astype('float32')

print("Optimización de merged_weather completada")
print(merged_weather.info(memory_usage='deep'))

merged_weather = merged_weather.drop('forecast_datetime', axis=1)


#9no paso , defino funciones y luego merge de a chunks por un tema de memoria

def process_weather_chunks(start_chunk, end_chunk):
    periods = [
        ('2021-09-01', '2021-10-01'), ('2021-10-01', '2021-11-01'), ('2021-11-01', '2021-12-01'), ('2021-12-01', '2022-01-01'),
        ('2022-01-01', '2022-02-01'), ('2022-02-01', '2022-03-01'), ('2022-03-01', '2022-04-01'), ('2022-04-01', '2022-05-01'),
        ('2022-05-01', '2022-06-01'), ('2022-06-01', '2022-07-01'), ('2022-07-01', '2022-08-01'), ('2022-08-01', '2022-09-01'),
        ('2022-09-01', '2022-10-01'), ('2022-10-01', '2022-11-01'), ('2022-11-01', '2022-12-01'), ('2022-12-01', '2023-01-01'),
        ('2023-01-01', '2023-02-01'), ('2023-02-01', '2023-03-01'), ('2023-03-01', '2023-04-01'), ('2023-04-01', '2023-05-01'),
        ('2023-05-01', '2023-05-31')
    ]
    
    for i in range(start_chunk, end_chunk):
        start_date, end_date = periods[i]
        print(f"Procesando período {i+1}: {start_date} a {end_date}")
        
        # Filtrar por período
        mask_train = (train4['datetime'] >= start_date) & (train4['datetime'] < end_date)
        train4_chunk = train4[mask_train].copy()
        
        mask_weather = (merged_weather['datetime'] >= start_date) & (merged_weather['datetime'] < end_date)
        weather_chunk = merged_weather[mask_weather].copy()
        
        # Merge y guardar
        chunk_result = pd.merge(train4_chunk, weather_chunk, on='datetime', how='left')
        chunk_result.to_parquet(f'train5_chunk_{i+1}.parquet', index=False)
        print(f"Chunk {i+1} guardado: {chunk_result.shape}")
        
        # Liberar memoria
        del train4_chunk, weather_chunk, chunk_result

# lo hice asi para no saturar
process_weather_chunks_corrected(0, 3)

process_weather_chunks_corrected(3, 9)

process_weather_chunks_corrected(9, 15)

process_weather_chunks_corrected(15, 21)


#junto todo

# Concatenar de a grupos para evitar saturar memoria
def concat_chunks_by_groups(group_size=1):
    group_files = []
    
    # Crear grupos intermedios
    for group_start in range(1, 22, group_size):
        group_end = min(group_start + group_size, 22)
        print(f"Procesando grupo: chunks {group_start} a {group_end-1}")
        
        # Leer chunks del grupo
        group_chunks = []
        for i in range(group_start, group_end):
            try:
                chunk = pd.read_parquet(f'train5_corrected_chunk_{i}.parquet')  # CORREGIDO
                group_chunks.append(chunk)
            except FileNotFoundError:
                print(f"Archivo train5_corrected_chunk_{i}.parquet no encontrado")
        
        # Concatenar grupo y guardar
        if group_chunks:
            group_df = pd.concat(group_chunks, ignore_index=True)
            group_filename = f'train5_group_{group_start}_{group_end-1}.parquet'
            group_df.to_parquet(group_filename, index=False)
            group_files.append(group_filename)
            print(f"Grupo guardado: {group_filename} - Shape: {group_df.shape}")
            
            # Liberar memoria
            del group_chunks, group_df
    
    # Ahora concatenar los grupos (serán muchos menos archivos)
    print("Concatenando grupos finales...")
    final_chunks = []
    for group_file in group_files:
        group_data = pd.read_parquet(group_file)
        final_chunks.append(group_data)
    
    train5 = pd.concat(final_chunks, ignore_index=True)
    return train5

# Ejecutar
train5 = concat_chunks_by_groups(1)
print(f"Dataset final: {train5.shape}")

#10mo paso , trabajo los problemas generados del merge

import gc

# 1. Identificar columnas a eliminar y renombrar SIN cargar datos
sample = train5.head(1000)  # Solo una muestra
cols_to_drop = [col for col in sample.columns if col.endswith('_y')]
rename_dict = {col: col[:-2] for col in sample.columns if col.endswith('_x')}
print(f"Eliminando {len(cols_to_drop)} columnas, renombrando {len(rename_dict)}")

# 2. Procesar por chunks de 1M filas
chunk_size = 1_000_000
chunks_processed = []

total_rows = len(train5)
n_chunks = (total_rows // chunk_size) + 1

for i in range(n_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, total_rows)
    
    print(f"Procesando chunk {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
    
    # Procesar chunk
    chunk = train5.iloc[start_idx:end_idx].copy()
    
    # Filtrar county != 12
    chunk = chunk[chunk['county'] != 12]
    
    if len(chunk) > 0:  # Solo si quedan filas
        # Eliminar columnas _y
        chunk = chunk.drop(columns=cols_to_drop)
        
        # Renombrar _x
        chunk.rename(columns=rename_dict, inplace=True)
        
        chunks_processed.append(chunk)
    
    # Limpiar memoria
    del chunk
    gc.collect()
    
    print(f"Memoria liberada, chunks guardados: {len(chunks_processed)}")

# 3. Concatenar chunks finales
print("Concatenando chunks finales...")
train5_clean = pd.concat(chunks_processed, ignore_index=True)

# 4. Limpiar
del chunks_processed, train5
gc.collect()

print(f"Dataset final: {train5_clean.shape[0]:,} filas y {train5_clean.shape[1]} columnas")

# Renombrar para usar el mismo nombre
train5 = train5_clean
del train5_clean
gc.collect()

# reordeno y veo que problemas siguen estando

# Orden específico que solicitaste
main_cols = [
    'row_id', 'target', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
    'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
    'longitude', 'lowest_price_per_mwh', 
    'highest_price_per_mwh', 'euros_per_mwh', 'eic_count', 'installed_capacity'
]

# Pares de clima: pronóstico (f) vs histórico - lado a lado para comparar
weather_pairs = [
    # Temperatura
    'ftemperature', 'temperature',
    
    # Punto de rocío  
    'fdewpoint', 'dewpoint',
    
    # Cobertura de nubes
    'fcloudcover_high', 'cloudcover_high',
    'fcloudcover_low', 'cloudcover_low', 
    'fcloudcover_mid', 'cloudcover_mid',
    'fcloudcover_total', 'cloudcover_total',
    
    # Viento
    'f10_metre_u_wind_component', 'windspeed_10m',
    'f10_metre_v_wind_component', 'winddirection_10m',
    
    # Radiación solar
    'fdirect_solar_radiation', 'direct_solar_radiation',
    'fsurface_solar_radiation_downwards', 'shortwave_radiation',
    'diffuse_radiation',  # solo histórico
    
    # Precipitación
    'fsnowfall', 'snowfall',
    'ftotal_precipitation', 'rain',
    
    # Otras variables climáticas
    'surface_pressure',  # solo histórico
    'weather_forecast_hour'
]

# Construir orden final
final_order = main_cols.copy()

# Agregar solo las columnas que existen
for col in weather_pairs:
    if col in train5.columns:
        final_order.append(col)

# Agregar cualquier columna restante
remaining_cols = [col for col in train5.columns if col not in final_order]
final_order.extend(remaining_cols)

print(f"Reordenando {len(final_order)} columnas...")
print("Pares clima encontrados:")
weather_cols_found = [col for col in weather_pairs if col in train5.columns]
for i in range(0, len(weather_cols_found), 2):
    pair = weather_cols_found[i:i+2]
    print(f"  {' vs '.join(pair)}")

# Reordenar
train5 = train5[final_order]

print("¡Columnas reordenadas!")
print(f"Primeras 10: {list(train5.columns[:10])}")
print(f"Clima inicia en posición: {final_order.index(weather_cols_found[0]) if weather_cols_found else 'N/A'}")

# re check de los na y lo hago de a chunks
from collections import defaultdict

# Procesar por chunks de 1M filas
chunk_size = 1_000_000
total_rows = len(train5)
n_chunks = (total_rows // chunk_size) + 1

# Variables para acumular resultados
missing_dates_count = defaultdict(int)
missing_by_county = defaultdict(int)
first_missing = None
last_complete = None
total_missing = 0

print(f"Procesando {total_rows:,} filas en {n_chunks} chunks...")

weather_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']

for i in range(n_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, total_rows)
    
    print(f"Chunk {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
    
    # Procesar chunk
    chunk = train5.iloc[start_idx:end_idx][['datetime', 'county'] + weather_cols]
    
    # Encontrar filas con NaN en clima
    missing_mask = chunk[weather_cols].isna().any(axis=1)
    missing_chunk = chunk[missing_mask]
    
    if len(missing_chunk) > 0:
        # Acumular conteos por fecha
        dates = missing_chunk['datetime'].dt.date.value_counts()
        for date, count in dates.items():
            missing_dates_count[date] += count
        
        # Acumular por county
        counties = missing_chunk['county'].value_counts()
        for county, count in counties.items():
            missing_by_county[county] += count
        
        # Tracking de primera fecha faltante
        chunk_first_missing = missing_chunk['datetime'].min()
        if first_missing is None or chunk_first_missing < first_missing:
            first_missing = chunk_first_missing
        
        total_missing += len(missing_chunk)
    
    # Tracking de última fecha completa
    complete_chunk = chunk[chunk['latitude'].notna()]
    if len(complete_chunk) > 0:
        chunk_last_complete = complete_chunk['datetime'].max()
        if last_complete is None or chunk_last_complete > last_complete:
            last_complete = chunk_last_complete
    
    # Limpiar memoria
    del chunk, missing_chunk
    gc.collect()
    
    if i % 5 == 0:  # cada 5 chunks
        print(f"  Faltantes encontrados hasta ahora: {total_missing:,}")

# Mostrar resultados
print(f"\n=== RESULTADOS ===")
print(f"Total filas con clima faltante: {total_missing:,}")
print(f"Total fechas afectadas: {len(missing_dates_count)}")

if missing_dates_count:
    # Convertir a Series para ordenar
    missing_dates_series = pd.Series(missing_dates_count).sort_index()
    
    print(f"\nPrimeras 10 fechas con más faltantes:")
    print(missing_dates_series.head(10))
    
    print(f"\nRango de fechas problemáticas:")
    print(f"Desde: {missing_dates_series.index.min()}")
    print(f"Hasta: {missing_dates_series.index.max()}")
    
    print(f"\nTop 5 counties con más faltantes:")
    county_series = pd.Series(missing_by_county).sort_values(ascending=False)
    print(county_series.head())
    
    print(f"\nTimeline:")
    print(f"Última fecha completa: {last_complete}")
    print(f"Primera fecha faltante: {first_missing}")
    if last_complete and first_missing:
        print(f"Gap de tiempo: {first_missing - last_complete}")
else:
    print("No se encontraron datos de clima faltantes!")

gc.collect()

## hago drop de los faltantes

# Eliminar filas con datos de clima faltantes
print(f"Dataset original: {len(train5):,} filas")

# Drop filas con NaN en clima
weather_key_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']
train5 = train5.dropna(subset=weather_key_cols)

print(f"Dataset limpio: {len(train5):,} filas")
print(f"Filas eliminadas: {32_965_048 - len(train5):,}")

# Verificar que no quedan NaN en clima
remaining_weather_nan = train5[weather_key_cols].isna().sum().sum()
print(f"NaN restantes en clima: {remaining_weather_nan}")

# Verificar fechas disponibles para el período de predicción
test_period = train5[
    (train5['datetime'].dt.date >= pd.to_datetime('2023-05-20').date()) &
    (train5['datetime'].dt.date <= pd.to_datetime('2023-05-31').date())
]['datetime'].dt.date.unique()

print(f"\nFechas disponibles cerca del período de test:")
print(sorted(test_period))

In [ ]:
###

In [ ]:
##################

In [ ]:
#################